In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
# import logging
# logging.basicConfig(filename='./test.log', level=logging.DEBUG, 
#                     format='%(asctime)s %(levelname)s %(name)s %(message)s')
# logger=logging.getLogger(__name__)

In [ ]:
# from IPython.core.magic import register_cell_magic

# @register_cell_magic('handle')
# def handle(line, cell):
#     try:
#         exec(cell)
#     except Exception as e:
#         logger.error(e)
#         raise # if you want the full trace-back in the notebook


# use %%handle when want to output error

In [ ]:
import time, os, fnmatch, shutil
t = time.localtime()
timestamp = time.strftime('%Y-%b-%d-%H-%M-%S', t)
print(timestamp)

# Co-training

For visualization of co-training process, we apply PCA to feature before training. This will make co-training process clear, but the result will be not accuracy because apply PCA will loss lots of information.

1. We assume only part of label exist

2. We only select binary case (Only when one name indicate two and only two author)

3. When we apply 10 fold with co-training, each fold of first iteration will be baseline compare to co-training

In [7]:
import os
import sys
import warnings

#warnings.filterwarnings('error')
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
np.set_printoptions(precision=4, suppress=True)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import com_func

#----- threshold for selecting set of name group -----------#
threshold_select_name_group = 100
#----- threshold for selecting min sample in name group ----#
threshold_lower = 100
threshold_upper = 110

apply_threshold_to_name_group_samples = True

pp_text = ["pv_dbow"]
pp_citation = ["n2v"]

Dataset = "pubmed"

# Basic Co-training details
1. Basic co-training algorithm required parameter p,n,k,u. Since we have 15 different dataset, we will assume p and n is 1, k is 30. (We are simulate real world situation where we do not know the distribution of unlabeled data amount 15 different dataset)
2. We set the parameter u as size of input train data (labeled+unlabeled) since our ublabeled data is not that large.
3. During co-training process, the confidence measure is using probability as confident score to evaluate whether unlabel sample should be label or not.
4. Probability only have few issues: 
Case 1: View diagreement in early iteration. The view could disagree with each other in early stage of co-training which will introduce noise; Case 2: 

In [8]:
import itertools
import warnings
import copy
import matplotlib.pyplot as plt
from matplotlib import lines
from adjustText import adjust_text
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score
from collections import defaultdict

# create co training classifier
class Co_training_clf(object):
    
    def __init__(self, clf1, clf2=None, u = 75, k=30):
        
        self.clf1 = clf1
        # assume co_training on one classifier
        if clf2 == None:
            self.clf2 = copy.deepcopy(clf1)
        else:
            self.clf2 = clf2
        # size of pool of unlabeled samples
        self.u = u
        # number of iteration
        self.k = k

    def softmax(self, x):
        """Compute softmax values for each sets of scores in x."""
        return np.exp(x) / np.sum(np.exp(x), axis=1)

    def sigmoid(self, x):
        return np.around(1 / (1 + np.exp(-x)), decimals=4).item()

    def init_L_U_U_prime(self, labels):
        # index of the samples that are initially labeled
        L = labels.index[labels["authorID"] != -1].tolist()
        # index of unlabeled samples
        U = labels.index[labels["authorID"] == -1].tolist()
        print("Initial L size: ", len(L))
        print("Initial U size: ", len(U))
        # random drawing sample from U
        random.shuffle(U)
        U_prime = U[-min(len(U), self.u):]
        # remove the samples in U_prime from U
        U = U[:-len(U_prime)]
        print("U size after drawing sample to U prime:",len(U))
        print("Initial U prime size: ", len(U_prime))
        return L, U, U_prime
    
    def check_iter_label_mapping(self, iter_clf1, iter_clf2):
        '''
        In theory, it shouldn't occur that label not mapping since it trained on same dataset but different view
        But add a check to make sure it won't occur and save the class mapping for later label unlabeled sample
        '''
        dv1_class_label = iter_clf1.classes_
        dv2_class_label = iter_clf2.classes_
        if all(dv1_class_label == dv2_class_label):
            self.classes_ = dv1_class_label
        else:
            sys.exit("Two view classifier label not mapping")

    def get_confidence_score(self, clf_h1, clf_h2, dv1, dv2):
        if hasattr(clf_h1, "predict_proba"):
            dv1_proba = clf_h1.predict_proba(dv1)
            dv2_proba = clf_h2.predict_proba(dv2)
        elif hasattr(clf_h1, "decision_function"):    # use decision function
            dv1_distance = np.array(clf_h1.decision_function(dv1))
            dv2_distance = np.array(clf_h2.decision_function(dv2))
            # if binary case, use sigmoid function to calculate probability
            if iter_train_label.nunique()==2:
                dv1_proba = []
                dv2_proba = []
                for distance in dv1_distance:
                    dv1_proba.append([1-self.sigmoid(distance), self.sigmoid(distance)])
                for distance in dv2_distance:
                    dv2_proba.append([1-self.sigmoid(distance), self.sigmoid(distance)])
            else:
                dv1_proba = self.softmax(dv1_distance)
                dv2_proba = self.softmax(dv2_distance)
            dv1_proba = np.array(dv1_proba)
            dv2_proba = np.array(dv2_proba)
            #print("Distance to hyperplane (dv1): ",dv1_distance)
            #print("Probability (dv1): ",dv1_proba)
        else:
            sys.exit("No confident score for process")
        
        return dv1_proba, dv2_proba

    def label_p_n_samples(self, proba, rank, proba_sample_idx_map):
        U_prime_size = len(proba)
        self_trained_sample_idx = []
        self_trained_confident = []
        for label, confident_rank in enumerate(rank):
            # 0 positive sample
            if label==0:
                p = []
                p_confident = []
                index = 0
                while(len(p) < self.p):
                    max_conf_sample_index = confident_rank[index]
                    # ---- if positive predict proba is more than 50% ------- #
                    if (proba[max_conf_sample_index][label] > 0.5):
                        #print('Sample idx: P: ', proba_sample_idx_map[max_conf_sample_index], " : ", proba[max_conf_sample_index])
                        p.append(proba_sample_idx_map[max_conf_sample_index])
                        p_confident.append(proba[max_conf_sample_index][label])
                    index +=1
                    if (index>=U_prime_size):
                        break
                self_trained_sample_idx.append(p)
                self_trained_confident.append(p_confident)
            # 1 negative sample
            elif label == 1:
                n = []
                n_confident = []
                index = 0
                while(len(n) < self.n):
                    max_conf_sample_index = confident_rank[index]
                    # ---- if negative predict proba is more than 50% ------- #
                    if (proba[max_conf_sample_index][label] > 0.5):
                        #print('Sample idx: N: ', proba_sample_idx_map[max_conf_sample_index], " : ", proba[max_conf_sample_index])
                        n.append(proba_sample_idx_map[max_conf_sample_index])
                        n_confident.append(proba[max_conf_sample_index][label])
                    index +=1
                    if (index>=U_prime_size):
                        break
                self_trained_sample_idx.append(n)
                self_trained_confident.append(n_confident)
            else:
                print("Class label error")
        return self_trained_sample_idx, self_trained_confident

    def get_self_labeled_sample(self):
        '''
        return:
            self-labeled new positive, self-labeled new negative (Index)
        '''
        return self.self_new_all, self.self_new_at_k

    def set_PCA(self, pca=None):
        self.pca = pca

    def fit(self, train_data, validation_data, dataset_name=None, plot_save_path=None):
        # sync input datatype
        for idx, item in enumerate(train_data):
            if not isinstance(item, pd.DataFrame):
                item = pd.DataFrame(item)
            item.reset_index(drop=True,inplace=True)
            train_data[idx] = item
        dv1 = train_data[0]
        dv2 = train_data[1]
        labels = train_data[-1]
        dv1_validation = validation_data[0]
        dv2_validation = validation_data[1] 
        label_validation = validation_data[-1]
        # using all unlabeled sample instead of pool of unlabeled sample
        self.u = len(labels)
        L_plot, U_plot, U_prime_plot = self.init_L_U_U_prime(labels)
        L, U, U_prime = ([] for i in range(3)) 
        #print("All data index: ",dv1.index.values)
        #print("L_plot: ", L_plot)
        #print("U_plot: ", U_plot)
        #print("U_prime_plot: ", U_prime_plot)
        
        # index of self labeled samples
        self.self_new_all = defaultdict(list)
        self.self_new_at_k = defaultdict(list)
        self.h1_new_idx = defaultdict(list)
        self.h2_new_idx = defaultdict(list)
        # when fit co-train, we collect f1 on validation samples wrt each iteration
        self.f1_on_validation_dv1 = []
        self.f1_on_validation_dv2 = []
        self.f1_on_validation_combined = []
        self.iterCounter = 0
        ''' train clf with concatenated features for k=0'''
        temp_d1 = dv1.iloc[L_plot]
        temp_d2 = dv2.iloc[L_plot]
        concatenate_data = pd.concat([temp_d1,temp_d2], axis=1, ignore_index=True)
        concatenate_vaildation = pd.concat([dv1_validation,dv2_validation], axis=1, ignore_index=True)
        concatenate_clf = copy.deepcopy(self.clf1)
        concatenate_clf.fit(concatenate_data, labels.iloc[L_plot].values.ravel())
        baseline_predict_result = concatenate_clf.predict(concatenate_vaildation)
        self.f1_baseline = f1_score(label_validation, baseline_predict_result, average='macro')
        # --------- plot PCA reduced plot for initial stage of train -------------- #
        pca = self.pca
        '''Notice pca will change input datatype dataframe to datatype list, keep it's order,
           or re-assign dv1.index.values to the output'''
        pca_dv1 = pca.fit_transform(X=dv1)
        pca_dv2 = pca.fit_transform(X=dv2)
        #loop until we have assigned labels to every sample in U and U_prime or we hit our iteration break condition
        while U_prime_plot:
            '''
            Extract labeled training sample from training sample and train classifier, then make prediction
            Evaluate performance of current iteration classifier
            '''
            # print("Iteration:",self.iterCounter, " L_plot: ",L_plot)
            # print("Iteration:",self.iterCounter, " U_prime_plot: ",U_prime_plot)
            # ------------- get labeled samples for train ----------- # 
            iter_train_d1 = dv1.iloc[L_plot]
            iter_train_d2 = dv2.iloc[L_plot]
            iter_train_label = labels.iloc[L_plot]
            # ----------- get U_prime unlabeled samples  ------------ #
            iter_unlabeled_d1 = dv1.iloc[U_prime_plot]
            iter_unlabeled_d2 = dv2.iloc[U_prime_plot]
            # ------------ train different view with classifier ----------- #
            iter_clf1 = copy.deepcopy(self.clf1) 
            iter_clf2 = copy.deepcopy(self.clf2)
            iter_clf1.fit(iter_train_d1, iter_train_label.values.ravel())
            iter_clf2.fit(iter_train_d2, iter_train_label.values.ravel())
            self.check_iter_label_mapping(iter_clf1, iter_clf2)
            #obtain input data class ratio
            temp = labels.iloc[L_plot]['authorID'].value_counts(normalize=True)
            data_ratio = []
            for classes in self.classes_:
                data_ratio.append(temp[classes]*10)
            data_ratio = [i/min(data_ratio) for i in data_ratio]
            self.p = int(data_ratio[0])
            self.n = int(data_ratio[1])
            print("P value: ", self.p, " N value: ", self.n)
            # ------------- test error on validation data --------------- #
            # make prediction on validation data
            y1 = iter_clf1.predict(dv1_validation)
            y2 = iter_clf2.predict(dv2_validation)
            # f1 score on each iteration
            f1_dv1 = f1_dv2 = 0
            f1_dv1 = f1_score(label_validation, y1, average='macro')
            f1_dv2 = f1_score(label_validation, y2, average='macro')
            # collect f1 for current iteration
            self.f1_on_validation_dv1.append(f1_dv1)
            self.f1_on_validation_dv2.append(f1_dv2)
            
            # ----------- collect the result of different k value ----------- #
            temp_d1 = dv1.iloc[L_plot]
            temp_d2 = dv2.iloc[L_plot]
            temp_validation = [dv1_validation,dv2_validation]
            
            temp_clf1 = copy.deepcopy(self.clf1) 
            temp_clf2 = copy.deepcopy(self.clf2)
            temp_clf1.fit(temp_d1, labels.iloc[L_plot].values.ravel())
            temp_clf2.fit(temp_d2, labels.iloc[L_plot].values.ravel())
            
            predict_result = self.combined_predict(temp_clf1,temp_clf2,temp_validation)
            f1_combined = f1_score(label_validation, predict_result, average='macro')
            self.f1_on_validation_combined.append(f1_combined)
            
            new_train_label = labels.iloc[L_plot]
            if len(self.h1_new_idx["index"])==0:
                print("Iteration 0, strat self label.")
            else:
                print("Iteration",self.iterCounter," h1 new: ",self.h1_new_idx["index"][-1]," probs: ", self.h1_new_idx["confident"][-1])
                print("Iteration",self.iterCounter," h2 new: ", self.h2_new_idx["index"][-1], " probs: ", self.h2_new_idx["confident"][-1])
            ''' 
            Notice here dv1_proba and dv2_proba's index is index for u' (Unlabeled data only)
            We use index of u' to find index (position) of data in U where U and L is all data index
            '''
            # rank class probabilities for unlabeled sample for it's confidence measure
            dv1_proba, dv2_proba = self.get_confidence_score(clf_h1=iter_clf1, clf_h2=iter_clf2, 
                                                             dv1=iter_unlabeled_d1, dv2=iter_unlabeled_d2)
            proba_sample_idx_map = iter_unlabeled_d1.index
            dv1_proba_rank = []
            dv2_proba_rank = []
            # proba1_rank[i] is label i's confidence measure
            for class_proba in dv1_proba.T:
                dv1_proba_rank.append((-class_proba).argsort())
            for class_proba in dv2_proba.T:
                dv2_proba_rank.append((-class_proba).argsort())
            # print(dv1_proba)
            # print(dv1_proba_rank)
            # print(dv2_proba)
            # print(dv2_proba_rank)
            # h1 classifier self label data
            h1_new_sample, h1_new_probs = self.label_p_n_samples(dv1_proba, dv1_proba_rank, proba_sample_idx_map)
            # h2 classifier
            h2_new_sample, h2_new_probs = self.label_p_n_samples(dv2_proba, dv2_proba_rank, proba_sample_idx_map)
            
            '''Case if h1's new class 1 is h2's new class 2
            Example: Iteration 45  h1 new:  [[95], [150]]  probs:  [[0.89], [0.82]]
                     Iteration 45  h2 new:  [[147], [95]]  probs:  [[0.89], [0.92]]
            '''
            for i, h1_class_new in enumerate(h1_new_sample):
                temp_pop_list = []
                for j, h1_item in enumerate(h1_class_new):
                    for k, h2_class_new in enumerate(h2_new_sample):
                        for l, h2_item in enumerate(h2_class_new):
                            # conflict
                            if h1_item == h2_item:
                                if h1_new_probs[i][j]> h2_new_probs[k][l]:
                                    h2_new_sample[k].pop(l)
                                    h2_new_probs[k].pop(l)
                                else:
                                    temp_pop_list.append(j)
                temp_pop_list = list(set(temp_pop_list))
                temp_pop_list.sort(reverse=True)
                for item_idx in temp_pop_list:
                    h1_new_sample[i].pop(item_idx)
                    h1_new_probs[i].pop(item_idx)
            # collect statistic for plot only (before remove self-labeled sample from u')
            iter_h1_for_plot = list(itertools.chain(*h1_new_sample))
            iter_h2_for_plot = list(itertools.chain(*h2_new_sample))
            iter_h1_prob = list(itertools.chain(*h1_new_probs))
            iter_h2_prob = list(itertools.chain(*h2_new_probs))
            
            self.h1_new_idx["index"].append(iter_h1_for_plot)
            self.h1_new_idx["confident"].append(iter_h1_prob)
            self.h2_new_idx["index"].append(iter_h2_for_plot)
            self.h2_new_idx["confident"].append(iter_h2_prob)
            '''
            Add most confidence samples as new training samples, auto label the samples and remove it from U_prime
            Special Case: if two view classifier give same most confident sample and p is 1, only 1 datapoint self-labeled
            '''
            roundNew = list(zip(h1_new_sample, h2_new_sample))
            roundNew_flatten_unique = []
            for label, round_new in enumerate(roundNew):
                round_new = set([item for sublist in round_new for item in sublist])
                round_new = [idx for idx in round_new]
                self.self_new_all[self.classes_[label]].append(round_new)
                roundNew_flatten_unique.extend(round_new)
                # add label to those new samples
                #print(labels["authorID"][round_new])
                labels["authorID"][round_new] = self.classes_[label]
                #print(labels["authorID"][round_new])
                #print(self.classes_[label]," (u' idx): ",round_new)
            #print(roundNew_flatten_unique)
            # extend the labeled sample
            L_plot.extend(roundNew_flatten_unique)
            # remove the labeled sample from U_prime
            U_prime_plot = [x for x in U_prime_plot if x not in roundNew_flatten_unique]
            #print(U_prime_plot)
            # randomly choice 2p+2n examples from u to replenish u_prime
            replenishItem = U_plot[-(2*self.p+2*self.n):]
            U_prime_plot.extend(replenishItem)
            U_plot = U_plot[:-len(replenishItem)]
            self.iterCounter +=1
            ''' --------- if reach loop condition k, we extract L from L_plot --------- '''
            if self.iterCounter <= self.k:
                L = copy.deepcopy(L_plot)
                U = copy.deepcopy(U_plot)
                U_prime = copy.deepcopy(U_prime_plot)
                self.self_new_at_k = copy.deepcopy(self.self_new_all)
            ''' # ----------- plot the last iteration of co-training process -------------- # '''
            if not U_prime_plot:
                self.iterCounter +=1
                # --------- test error on validation data --------------------- #
                # make prediction on validation data
                y1 = iter_clf1.predict(dv1_validation)
                y2 = iter_clf2.predict(dv2_validation)
                # f1 score on each iteration
                f1_dv1 = f1_dv2 = 0
                f1_dv1 = f1_score(label_validation, y1, average='macro')
                f1_dv2 = f1_score(label_validation, y2, average='macro')
                # collect f1 for current iteration
                self.f1_on_validation_dv1.append(f1_dv1)
                self.f1_on_validation_dv2.append(f1_dv2)
                
                # ----------- collect the result of different k value ----------- #
                temp_d1 = dv1.iloc[L_plot]
                temp_d2 = dv2.iloc[L_plot]
                temp_validation = [dv1_validation,dv2_validation]

                temp_clf1 = copy.deepcopy(self.clf1) 
                temp_clf2 = copy.deepcopy(self.clf2)
                temp_clf1.fit(temp_d1, labels.iloc[L_plot].values.ravel())
                temp_clf2.fit(temp_d2, labels.iloc[L_plot].values.ravel())

                predict_result = self.combined_predict(temp_clf1,temp_clf2,temp_validation)
                f1_combined = f1_score(label_validation, predict_result, average='macro')
                self.f1_on_validation_combined.append(f1_combined)
                
                new_train_label = labels.iloc[L_plot]
                print("Iteration",self.iterCounter," h1 new: ",self.h1_new_idx["index"][-1]," probs: ", self.h1_new_idx["confident"][-1])
                print("Iteration",self.iterCounter," h2 new: ", self.h2_new_idx["index"][-1], " probs: ", self.h2_new_idx["confident"][-1])
        
        selected_return_iter = self.k
        if self.iterCounter <= self.k:
            selected_return_iter = self.iterCounter
            self.k=self.iterCounter
        print(self.k,"iterations total Labeled number: ", len(L), " Still unlabeled number: ", len(U_prime))
        print(self.iterCounter,"iteration total Labeled number: ", len(L_plot), " Still unlabeled number: ", len(U_prime_plot))
        counter = Counter(L_plot)
        print(counter.most_common(3))
        if len(set(L_plot)) == len(L_plot):
            print("success")
        else:
            print("duplicate found")
            sys.exit("error")
        #print(self.f1_on_validation_dv1)
        #print(self.f1_on_validation_dv2)
        # final train
        newtrain_d1 = dv1.iloc[L]
        newtrain_d2 = dv2.iloc[L]
        self.clf1.fit(newtrain_d1, labels.iloc[L].values.ravel())
        self.clf2.fit(newtrain_d2, labels.iloc[L].values.ravel())
        '''
        Evalutation plot for co-training process, save f1 score vs number of iteration plot
        '''
        if (dataset_name != None) and (plot_save_path != None):
            # ----------------- plot h1/h2 result separately -------------------- #
            default_text_based = [self.f1_on_validation_dv1[0]] * self.iterCounter
            default_citation_based = [self.f1_on_validation_dv2[0]] * self.iterCounter
            co_train_text_based = self.f1_on_validation_dv1
            co_train_citation_based = self.f1_on_validation_dv2
            step = np.arange(start=0, stop=self.iterCounter, step=1)
            
            #print("step:",len(step))
            #print("default:",self.iterCounter)
            #print("co-training v1:",len(co_train_text_based))
            #print("co-training v2:",len(co_train_citation_based))
            #print("default concatenate:",self.iterCounter)
            #print("default combined:",self.iterCounter)
            #print("co-training combined:",len(self.f1_on_validation_combined))

            fig = plt.figure(figsize=(8,6))
            ax = plt.axes()
            plt.plot(step, default_text_based, linestyle='dashed', label="Text based default")
            plt.plot(step, default_citation_based, linestyle='dashdot', label="Citation based default")
            plt.plot(step, co_train_text_based, linestyle='solid', marker = "*", label="Text based")
            plt.plot(step, co_train_citation_based, linestyle='dotted', marker = "+", label="Citation based")
            handles, _  = ax.get_legend_handles_labels()
            plt.axvline(x=(selected_return_iter-1), color='r', label="k")
            _, labels = ax.get_legend_handles_labels()
            vertical_line = lines.Line2D([], [],  marker='|', linestyle='None', color="r", markersize=10, markeredgewidth=1.5)
            handles.append(vertical_line)
            ax.autoscale_view()
            legend = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            f1_vs_iteration_plot_name = plot_save_path+dataset_name+"_co_train_iteration_f1(1).png"
            counter = 1
            while os.path.exists(plot_save_path+dataset_name+"_co_train_iteration_f1(%s).png" % counter):
                counter+=1
                f1_vs_iteration_plot_name = plot_save_path+dataset_name+"_co_train_iteration_f1(%s).png" % counter
            plt.savefig(f1_vs_iteration_plot_name, dpi=150, bbox_extra_artists=(legend,), bbox_inches='tight')
            # plt.show()
            plt.close("all")
            
            # -------------------- plot combined result ------------------------- #
            default_concatenate = [self.f1_baseline] * self.iterCounter
            default_combined = [self.f1_on_validation_combined[0]] * self.iterCounter
            co_train_combined = self.f1_on_validation_combined
            fig = plt.figure(figsize=(8,6))
            ax = plt.axes()
            plt.plot(step, default_concatenate, linestyle='dashed', label="Concatenate default")
            plt.plot(step, default_combined, linestyle='dotted', label="Combined default")
            plt.plot(step, co_train_combined, linestyle='solid', marker = "*", label="Combined co-train")
            handles, _  = ax.get_legend_handles_labels()
            plt.axvline(x=(selected_return_iter-1), color='r', label="k")
            _, labels = ax.get_legend_handles_labels()
            vertical_line = lines.Line2D([], [],  marker='|', linestyle='None', color="r", markersize=10, markeredgewidth=1.5)
            handles.append(vertical_line)
            ax.autoscale_view()
            legend = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            f1_vs_iteration_plot_name = plot_save_path+dataset_name+"_co_train_combined_iteration_f1(1).png"
            counter = 1
            while os.path.exists(plot_save_path+dataset_name+"_co_train_combined_iteration_f1(%s).png" % counter):
                counter+=1
                f1_vs_iteration_plot_name = plot_save_path+dataset_name+"_co_train_combined_iteration_f1(%s).png" % counter
            plt.savefig(f1_vs_iteration_plot_name, dpi=150, bbox_extra_artists=(legend,), bbox_inches='tight')
            # plt.show()
            plt.close("all")
        return "Training Done"

    def co_train_process_f1(self):
        return self.f1_on_validation_dv1, self.f1_on_validation_dv2, self.f1_on_validation_combined, self.f1_baseline

    def get_iter_count(self):
        return self.iterCounter
    
    def get_k(self):
        return self.k

    def predict(self, data):
        dv1=data[0]
        dv2=data[1]
        dv1_predict = self.clf1.predict(dv1)
        dv2_predict = self.clf2.predict(dv2)
        #fill pred with -1 so we can identify the samples in which error occur
        y_pred = ["-1"] * dv1.shape[0]
        for i, (dv1_y, dv2_y) in enumerate(zip(dv1_predict, dv2_predict)):
            # if both agree on label
            if dv1_y == dv2_y:
                y_pred[i] = dv1_y
            # If disagree on label and support proba method: We times probability together, choice class have higher probabilities
            elif hasattr(self.clf1, "predict_proba") and hasattr(self.clf2, "predict_proba"):
                h1_probas = self.clf1.predict_proba([dv1.iloc[i]])[0]
                h2_probas = self.clf2.predict_proba([dv2.iloc[i]])[0]
                final_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(h1_probas, h2_probas)]
                #print("h1 and h2 disagree on",i, " h1 Proba : ",h1_probas, " h2 Proba: ", h2_probas)
                #print("product probas:",final_y_probas)
                max_prob_idx = final_y_probas.index(max(final_y_probas))
                y_pred[i] = self.classes_[max_prob_idx]
                #print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            # If disagree on label and not support proba method: We use decision_function to first calculate probability,
            # then we times calculate probability together, choice class have higher probabilities
            elif hasattr(self.clf1, "decision_function") and hasattr(self.clf2, "decision_function"):
                dv1_distance = self.clf1.decision_function([dv1.iloc[i]])
                dv2_distance = self.clf2.decision_function([dv2.iloc[i]])
                if len(self.clf1.classes_)==2:
                    h1_probas = np.array([1-self.sigmoid(dv1_distance), self.sigmoid(dv1_distance)])
                    h2_probas = np.array([1-self.sigmoid(dv2_distance), self.sigmoid(dv2_distance)])
                else:
                    h1_probas = self.softmax(dv1_distance)
                    h2_probas = self.softmax(dv2_distance)
                final_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(h1_probas, h2_probas)]
                #print("h1 and h2 disagree on",i, " h1 Proba : ",h1_probas, " h2 Proba: ", h2_probas)
                #print("product probas:",final_y_probas)
                max_prob_idx = final_y_probas.index(max(final_y_probas))
                y_pred[i] = self.classes_[max_prob_idx]
                #print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            # disagree and not support any confidence measure
            else:
                sys.exit("Error occur")

        # convert final result to np array
        y_pred_np_array = np.asarray(y_pred)
        return y_pred_np_array

    def predict_proba(self, dv1, dv2):
        # the predicted probabilities is simply a product (*) of probabilities given from each classifier trained
        h1_probas = self.clf1.predict_proba(dv1)
        h2_probas = self.clf2.predict_proba(dv2)
        
        proba = (h1_probas*h2_probas)
        return proba
    
    ''' ------------- function for optimize parameter k only ---------------- '''
    def combined_predict(self, dv1_clf, dv2_clf, data):
        dv1=data[0]
        dv2=data[1]
        dv1_predict = dv1_clf.predict(dv1)
        dv2_predict = dv2_clf.predict(dv2)
        #fill pred with -1 so we can identify the samples in which error occur
        y_pred = ["-1"] * dv1.shape[0]
        for i, (dv1_y, dv2_y) in enumerate(zip(dv1_predict, dv2_predict)):
            # if both agree on label
            if dv1_y == dv2_y:
                y_pred[i] = dv1_y
            # If disagree on label and support proba method: We times probability together, choice class have higher probabilities
            elif hasattr(dv1_clf, "predict_proba") and hasattr(dv2_clf, "predict_proba"):
                h1_probas = dv1_clf.predict_proba([dv1.iloc[i]])[0]
                h2_probas = dv2_clf.predict_proba([dv2.iloc[i]])[0]
                final_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(h1_probas, h2_probas)]
                #print("h1 and h2 disagree on",i, " h1 Proba : ",h1_probas, " h2 Proba: ", h2_probas)
                #print("product probas:",final_y_probas)
                max_prob_idx = final_y_probas.index(max(final_y_probas))
                y_pred[i] = self.classes_[max_prob_idx]
                #print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            # If disagree on label and not support proba method: We use decision_function to first calculate probability,
            # then we times calculate probability together, choice class have higher probabilities
            elif hasattr(dv1_clf, "decision_function") and hasattr(dv2_clf, "decision_function"):
                dv1_distance = dv1_clf.decision_function([dv1.iloc[i]])
                dv2_distance = dv2_clf.decision_function([dv2.iloc[i]])
                if len(dv1_clf.classes_)==2:
                    h1_probas = np.array([1-self.sigmoid(dv1_distance), self.sigmoid(dv1_distance)])
                    h2_probas = np.array([1-self.sigmoid(dv2_distance), self.sigmoid(dv2_distance)])
                else:
                    h1_probas = self.softmax(dv1_distance)
                    h2_probas = self.softmax(dv2_distance)
                final_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(h1_probas, h2_probas)]
                #print("h1 and h2 disagree on",i, " h1 Proba : ",h1_probas, " h2 Proba: ", h2_probas)
                #print("product probas:",final_y_probas)
                max_prob_idx = final_y_probas.index(max(final_y_probas))
                y_pred[i] = self.classes_[max_prob_idx]
                #print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            # disagree and not support any confidence measure
            else:
                sys.exit("Error occur")

        # convert final result to np array
        y_pred_np_array = np.asarray(y_pred)
        return y_pred_np_array


# Improved co-training

Possible improvement:

Part 1: Parameter input
1. Using two different algorithm for view one and view two.
2. Instead of using parameter k to control number of iteration, using a stop criterion where if unlabeled sample can't get more than 95% of confidence on their confidence score, then stop iteration and finish co-training.
3. Parameter p, n are not needed, use input data class ratio replace it (unlabeled distribution may different from labeled, this is unlikely works in reality).
4. u prime not needed, use all unlabeled data.

Part 2: Change confident measure method
1. Changes method used when determine which class unlabeled sample belones to, instead of using confident score, we could use classifiers saved during each iteration of training to perform an majority voting. (Need to careful about number of iteration in co-training, since adding bad unlabel data as label data is easy to occur) (Source: 2004_Co-training and Self-training for Word Sense Disambiguation)

Part 3: Change details in confident score method
1. Add an third classifier and only train with original labeled samples(no-self-labeled sample), then use third classifier to evaluate self-labeled samples and get it's confidence score. Combine three classifier's confidence score together to get final decision.
2. COTRADE method: construct undirected neighborhood graph using all samples, after finding most confident sample with it's corresponding class. Then, calculate distance between all labeled samples in this class and most confident sample in this class. This distance from graph + probability from supervised algorithm will be used together as confidence score.

Part 4: Add checking to reduce the chance of adding mislabeled data
1. Add an check on validation after new label sample is added, if not improving h1/h2, remove new labeled sample.

Part 5: Change algorithm and make it work for muti-class
1. Using same concept of muti-class SVM, train many OVR binary classifier. (To expensive)
2. Allow algorithm directly take muti-class case and using same concept as binary co-training.


My idea:
1. Use one classifier (check)
2. Have parameter k as default, but can be stoped early (check)
3. Parameter p, n are replace with sl_base which set as input data class ratio*sl_base. (Allow muti-class case and will maintaining the class distribution in L. However, unlabeled distribution may different from labeled, this is unlikely works in reality) (check)
4. u prime not needed, use all unlabeled data. (check)
5. Adopt filter view disagreed samples, instead directly remove disagreed sample, we will give a score to evaluate disagreement between views and add this score as part of confident score. (2008-Multi-view learning in the presence of view disagreement)
6. We will accumulate the probability of first iteration and current iteration; add them together as part of confident score. (check)
7. Adopt COTRADE method, their assumption where that a correctly labeled example should be very close to samples in L with corresponding label same to be very useful. Thus a k-mean clustering is train with labeled, then transform unlabel data into same instance space as labeled samples, then calculate distance to each cluster's centroid, then use distance perfrom an softmax to get reverse of probability of sample belone to class. After that 1- reversed probability get probability and add it as part of co-train confident score. (check)
8. We can think confident score as quality of unlabeled data, after each co-training iteration, it should always increase in best case. However, when we see a decrease in confident score compare to pervious iteration, it means the data quality level have decresed, therefore we need to check whether heighest probability of samples is greater than 0.8 or not. 


9. Notice that our main evaluator for whether a new sample should be added to L is current iteration probability, not the confident score, confident score is only used for sample quality check. (check)
10. When no sample been added for both view, we stop co-training iteration since data left may provide more noise than information. (check)

In [9]:
import itertools
import warnings
import copy
import math
import matplotlib.pyplot as plt
from adjustText import adjust_text
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score

from collections import defaultdict

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

# create co training classifier
class Improved_co_training_clf(object):
    
    def __init__(self, clf=[], view_num=2, k=30, **args):
        if len(clf) == view_num:
            self.clf = clf
        elif len(clf) == 1:
            self.clf = [copy.deepcopy(clf[0]) for i in range(view_num)]
        else:
            sys.exit("Classifier doesn't match with view number.")
        self.view_num = view_num
        self.iter_new_size =[]
        # number of iteration
        self.k = k

    def softmax(self, x):
        """Compute softmax values for each sets of scores in x."""
        return np.exp(x) / np.sum(np.exp(x), axis=0)

    def sigmoid(self, x):
        return np.around(1 / (1 + np.exp(-x)), decimals=4).item()

    def init_L_U_U_prime(self, labels):
        # index of the samples that are initially labeled
        L = labels.index[labels["authorID"] != -1].tolist()
        # index of unlabeled samples
        U = labels.index[labels["authorID"] == -1].tolist()
        print("Initial L size: ", len(L))
        print("Initial U size: ", len(U))
        # random drawing sample from U
        random.shuffle(U)
        U_prime = U[-min(len(U), self.u):]
        # remove the samples in U_prime from U
        U = U[:-len(U_prime)]
        print("U size after drawing sample to U prime:",len(U))
        print("Initial U prime size: ", len(U_prime))
        return L, U, U_prime
    
    def check_iter_label_mapping(self, iter_clf):
        '''
        In theory, it shouldn't occur that label not mapping since it trained on same dataset but different view
        But add a check to make sure it won't occur and save the class mapping for later label unlabeled sample
        '''
        class_label = defaultdict(list)
        for idx,clf in enumerate(iter_clf):
            if idx ==0:
                self.classes_ = clf.classes_
            
            if not all(self.classes_ == clf.classes_):
                sys.exit("Two view classifier label not mapping")
        #print(self.classes_)
        return "checked"

    def get_confidence_score(self, clf, data):
        if hasattr(clf, "predict_proba"):
            proba = clf.predict_proba(data)
        elif hasattr(clf, "decision_function"):    # use decision function
            all_distance = np.array(clf.decision_function(data))
            # if binary case, use sigmoid function to calculate probability
            if iter_train_label.nunique()==2:
                proba = []
                for distance in all_distance:
                    proba.append([1-self.sigmoid(distance), self.sigmoid(distance)])
            else:
                proba = self.softmax(dv1_distance)
            proba = np.array(proba)
            #print("Distance to hyperplane: ",distance)
            #print("Probability: ",proba)
        else:
            sys.exit("No confident score for process")
        
        return proba

    def get_cluster_distance_as_proba(self, label_train, iter_train_label, unlabeled):
        kmeans = KMeans(n_clusters=len(self.classes_)).fit(label_train)
        # map correct idx to label
        kmeans_predict = kmeans.labels_
        label_mapping = list(zip(iter_train_label, kmeans_predict))
        
        lmc = Counter(label_mapping)
        final_label_mapping = []
        for label in np.unique(iter_train_label):
            for (item,index),count in lmc.most_common():
                if item == label:
                    #print("label:",label, " idx:",index, " count:",count)
                    final_label_mapping.append((item,index))
                    break
        #print(final_label_mapping)
        
        # convert distance to centroid to probability of belone to class
        dist_to_centroid = kmeans.transform(unlabeled)
        cluster_proba = []
        for distance in dist_to_centroid:
            reverse_proba = self.softmax(distance)
            proba = [1-x for x in reverse_proba]
            cluster_proba.append(proba)
        cluster_proba = np.array(cluster_proba)
        
        # map the proba same order as train in supervised learning
        new_permutation = [idx for label, idx in final_label_mapping]
        #print(new_permutation)
        idx = np.empty_like(new_permutation)
        idx[new_permutation] = np.arange(len(new_permutation))
        cluster_proba[:] = cluster_proba[:, idx]
        
        return cluster_proba

    def label_samples(self, score, rank, score_sample_idx_map,curr_iter_proba):
        U_prime_size = len(score)
        self_trained_sample_idx = []
        self_trained_confident = []
        self_trained_iter_proba = []
        self.iter_new_size = defaultdict(list)

        for class_idx in range(len(self.classes_)):
            self.iter_new_size[self.classes_[class_idx]].append(int(self.class_ratio[class_idx]))
        #print(self.iter_new_size)
        
        '''Search for samples that can be labeled'''
        for label, confident_rank in enumerate(rank):
            # ---------- find heightest current iteration probability for all class ----- #
            class_curr_iter_proba = [row[label] for row in curr_iter_proba]
            class_curr_iter_proba.sort()
            class_heightest_proba = max(class_curr_iter_proba)
            # ------------------- find last iteration max confidence score -------------- #
            pervious_iter_confidence_score = self.self_new_all[self.classes_[label]+"_score"]
            if len(pervious_iter_confidence_score)==0:
                last_iter_max_confidence_score=0
            else:
                last_iter_max_confidence_score = max(self.self_new_all[self.classes_[label]+"_score"][-1], default=0)
            
            new_label_sample_size = [size for size in self.iter_new_size[self.classes_[label]]][0]
            new_sample_idx = []
            new_sample_confident_score = []
            new_sample_proba = []
            index = 0
            while(len(new_sample_idx) < new_label_sample_size):
                max_conf_sample_index = confident_rank[index]
                #print('Select idx: ', max_conf_sample_index,":",score_sample_idx_map[max_conf_sample_index]," have score: ",score[max_conf_sample_index], " probability ",curr_iter_proba[max_conf_sample_index])
                if (score_sample_idx_map[max_conf_sample_index]<last_iter_max_confidence_score) and (class_heightest_proba<0.8):
                    print('Most confident idx: ', max_conf_sample_index,":",score_sample_idx_map[max_conf_sample_index]," have score: ",score[max_conf_sample_index]," smaller than last iteration max score: ",last_iter_max_confidence_score," probability ",curr_iter_proba[max_conf_sample_index], " no bigger than 0.8, sample not used")
                    break
                elif curr_iter_proba[max_conf_sample_index][label] <0.5:
                    #print('Most confident idx: ', max_conf_sample_index,":",score_sample_idx_map[max_conf_sample_index]," have score: ",score[max_conf_sample_index], "have probability ",curr_iter_proba[max_conf_sample_index], " not in top 3, thus sample not used")
                    index +=1
                else:
                    #print('Class:', self.classes_[label],' idx: ',max_conf_sample_index,":",score_sample_idx_map[max_conf_sample_index], " score: ", score[max_conf_sample_index], " iter proba: ", curr_iter_proba[max_conf_sample_index])
                    new_sample_idx.append(score_sample_idx_map[max_conf_sample_index])
                    new_sample_confident_score.append(score[max_conf_sample_index][label])
                    new_sample_proba.append(curr_iter_proba[max_conf_sample_index][label])
                    index +=1
                if (index>=U_prime_size) :
                    break
            self_trained_sample_idx.append(new_sample_idx)
            self_trained_confident.append(new_sample_confident_score)
            self_trained_iter_proba.append(new_sample_proba)
        return self_trained_sample_idx, self_trained_confident, self_trained_iter_proba

    def get_self_labeled_sample(self):
        '''
        return:
            self-labeled new positive, self-labeled new negative (Index)
        '''
        temp = defaultdict(list)
        temp_k = defaultdict(list)
        for label in self.classes_:
            temp[label]=self.self_new_all[label]
            temp_k[label] = self.self_new_at_k[label]
        
        return temp, temp_k

    def set_PCA(self, pca=None):
        self.pca = pca

    def fit(self, train_data, validation_data, dataset_name=None, plot_save_path=None):
        # sync input datatype
        for idx, item in enumerate(train_data):
            if not isinstance(item, pd.DataFrame):
                item = pd.DataFrame(item)
            item.reset_index(drop=True,inplace=True)
            train_data[idx] = item
        
        labels = train_data[-1]
        label_validation = validation_data[-1]
        
        # using all unlabeled sample instead of pool of unlabeled sample
        self.u = len(labels)
        L_plot, U_plot, U_prime_plot = self.init_L_U_U_prime(labels)
        L, U, U_prime = ([] for i in range(3))
        #print("All data index: ",train_data[0].index.values)
        #print("L_plot: ", L_plot)
        #print("U_plot: ", U_plot)
        #print("U_prime_plot: ", U_prime_plot)
        
        # index of self labeled samples
        self.self_new_all = defaultdict(list)
        self.self_new_at_k = defaultdict(list)
        self.new_labeled_details = defaultdict(list)
        self.f1_on_validation = defaultdict(list)
        self.f1_on_validation_combined = []
        ''' train clf with concatenated features for k=0'''
        temp_train = defaultdict(list)
        for view_idx in range(self.view_num):
            temp_train[view_idx] = train_data[view_idx].iloc[L_plot]
        concatenate_data = pd.concat([temp_train[0],temp_train[1]], axis=1, ignore_index=True)
        concatenate_vaildation = pd.concat([validation_data[0],validation_data[1]], axis=1, ignore_index=True)
        concatenate_clf = copy.deepcopy(self.clf[0])
        concatenate_clf.fit(concatenate_data, labels.iloc[L_plot].values.ravel())
        baseline_predict_result = concatenate_clf.predict(concatenate_vaildation)
        self.f1_baseline = f1_score(label_validation, baseline_predict_result, average='macro')
        # save clf in each iteration
        self.all_iter_clf = []
        # when fit co-train, we collect f1 on validation samples wrt each iteration
        for view_idx in range(self.view_num):
            self.f1_on_validation["dv"+str(view_idx)]=[]
        self.iterCounter = 0
        # --------- plot PCA reduced plot for initial stage of train -------------- #
        pca = self.pca
        '''Notice pca will change input datatype dataframe to datatype list, keep it's order,
           or re-assign dv1.index.values to the output'''
        pca_dv1 = pca.fit_transform(X=train_data[0])
        pca_dv2 = pca.fit_transform(X=train_data[1])
        #loop until we have assigned labels to every sample in U and U_prime
        while U_prime_plot:
            '''
            Extract labeled training sample from training sample and train classifier, then make prediction
            Evaluate performance of current iteration classifier
            '''
            # print("Iteration:",self.iterCounter, " L_plot: ",L_plot)
            # print("Iteration:",self.iterCounter, " U_prime_plot: ",U_prime_plot)
            iter_train = defaultdict(list)
            iter_unlabeled = defaultdict(list)
            iter_clf = copy.deepcopy(self.clf) 
            iter_train_label = labels.iloc[L_plot]
            for view_idx in range(self.view_num):
                iter_train["dv"+str(view_idx)]=train_data[view_idx].iloc[L_plot]
                iter_unlabeled["dv"+str(view_idx)]=train_data[view_idx].iloc[U_prime_plot]
                
                iter_clf[view_idx].fit(iter_train["dv"+str(view_idx)], iter_train_label.values.ravel())

            self.check_iter_label_mapping(iter_clf)
            # ----------------- obtain input data class ratio --------------- #
            temp = labels.iloc[L_plot]['authorID'].value_counts(normalize=True)
            self.class_ratio = []
            for classes in self.classes_:
                self.class_ratio.append(temp[classes]*10)
            self.class_ratio = [i/min(self.class_ratio) for i in self.class_ratio]
            # ---------------------- test error on validation data --------------------- #
            y_predict = defaultdict(list)
            f1 = defaultdict(list)
            for view_idx in range(self.view_num):
                y_predict[view_idx]=iter_clf[view_idx].predict(validation_data[view_idx])
                f1["dv"+str(view_idx)]=f1_score(label_validation,y_predict[view_idx],average='macro')
                self.f1_on_validation["dv"+str(view_idx)].append(f1["dv"+str(view_idx)])
            
            self.all_iter_clf.append((self.iterCounter,iter_clf))
                        
            # ---------------- collect the result of different k value ----------------- #
            # --------------- collect result for concatenated features ----------------- #
            temp_train = defaultdict(list)
            temp_clf = defaultdict(list)
            for view_idx in range(self.view_num):
                temp_train[view_idx] = train_data[view_idx].iloc[L_plot]
                temp_clf[view_idx] = copy.deepcopy(self.clf[view_idx])
                temp_clf[view_idx].fit(temp_train[view_idx], labels.iloc[L_plot].values.ravel())
            
            predict_result = self.combined_predict(temp_clf[0],temp_clf[1],validation_data)
            f1_combined = f1_score(label_validation, predict_result, average='macro')
            self.f1_on_validation_combined.append(f1_combined)
            
            # ----------------------- start print details ------------------------ #
            new_train_label = labels.iloc[L_plot]
            if len(self.new_labeled_details["h0_new_idx"])==0:
                print("Iteration 0, strat self label.")
            else:
                print("Iteration",self.iterCounter," h1 new: ", self.new_labeled_details["h0_new_idx"][-1], " score: ", self.new_labeled_details["h0_new_proba"][-1])
                print("Iteration",self.iterCounter," h2 new: ", self.new_labeled_details["h1_new_idx"][-1], " score: ", self.new_labeled_details["h1_new_proba"][-1])
            
            '''
            Confidence measure:
            Part 1: View agreement score is probability of all view multiply together
            Case 1: Two view agree with very large confidence score (check)
            Case 2: Two view disagree, one view have large confidence, scoring towards large confident view's class (check)
            Case 3: Two view disagree and both have large confidence score
            Case 4: Two view agree/disagree with small confidence score
            '''
            unlabeled_pred_proba = []
            for view_idx in range(self.view_num):
                view_pred_proba = self.get_confidence_score(clf=iter_clf[view_idx],data=iter_unlabeled["dv"+str(view_idx)])
                unlabeled_pred_proba.append(view_pred_proba)
            view_agreement_score = np.array([np.multiply(x[0],x[1]) for x in zip(*unlabeled_pred_proba)])
            #print(unlabeled_pred_proba)
            #print(view_agreement_score)
            ''' 
            # Part 2: Accumulate predicted probability on each iteration as score
            '''
            diff_iter_score = defaultdict(list)
            curr_iter_proba = defaultdict(list)
            final_score = defaultdict(list)
            cluster_proba = defaultdict(list)
            score_rank = defaultdict(list)
            proba_sample_idx_map=iter_unlabeled["dv0"].index
            for view_idx in range(self.view_num):
                # ------ Part 1: Accumulate predicted probability on first and current iteration as score ------ #
                for iteration, clf in self.all_iter_clf:
                    if (iteration ==0) or (iteration ==self.iterCounter):
                        diff_iter_score[view_idx].append(self.get_confidence_score(clf=clf[view_idx], data=iter_unlabeled["dv"+str(view_idx)]))
                # add score in each iteration together
                final_score["dv"+str(view_idx)] = np.array([sum(x) for x in zip(*diff_iter_score[view_idx])])
                # save last iteration probability
                curr_iter_proba["dv"+str(view_idx)] = diff_iter_score[view_idx][-1]
                
                # ----------- Part 2: Incorporate clustering distance as part of score ------------ #
                cluster_proba[view_idx] = self.get_cluster_distance_as_proba(label_train=iter_train["dv"+str(view_idx)], iter_train_label=iter_train_label["authorID"],
                                                                             unlabeled=iter_unlabeled["dv"+str(view_idx)])
                final_score["dv"+str(view_idx)]=np.array([sum(x) for x in zip(final_score["dv"+str(view_idx)],cluster_proba[view_idx])])
                
                # ----------------- Part 3: Incorporate view agreement score ---------------------- #
                final_score["dv"+str(view_idx)]=np.array([sum(x) for x in zip(final_score["dv"+str(view_idx)],view_agreement_score)])
                
                # -------------- Part 4: Rank confidence and start self labeling ------------------ #
                # proba1_rank[i] is label i's confidence measure
                for class_proba in final_score["dv"+str(view_idx)].T:
                    score_rank["dv"+str(view_idx)].append((-class_proba).argsort())
                
            #print("All: ", diff_iter_score)
            #print("Current: ",curr_iter_proba)
            #print("Cluster proba: ", cluster_proba)
            #print("Final score: ",final_score)
            #print("Score rank: ", score_rank)
            
            ''' Start labelling '''
            new_sample = defaultdict(list)
            new_sample_score = defaultdict(list)
            new_sample_proba = defaultdict(list)
            flatten_new_idx = defaultdict(list)
            flatten_score = defaultdict(list)
            flatten_proba = defaultdict(list)
            self.new_labeled_details["Iteration"].append(self.iterCounter)
            
            for view_idx in range(self.view_num):
                (new_sample["dv"+str(view_idx)], 
                 new_sample_score["dv"+str(view_idx)], 
                 new_sample_proba["dv"+str(view_idx)]) = self.label_samples(final_score["dv"+str(view_idx)], 
                                                                            score_rank["dv"+str(view_idx)], 
                                                                            proba_sample_idx_map, 
                                                                            curr_iter_proba["dv"+str(view_idx)])
            '''Case if h1's new class 1 is h2's new class 2
            Example: Iteration 45  h1 new:  [[95], [150]]  probs:  [[0.89], [0.82]]
                     Iteration 45  h2 new:  [[147], [95]]  probs:  [[0.89], [0.92]]
            '''
            #print("before fix: ",new_sample)
            #print("before fix: ",new_sample_proba)
            full_views_list = list(new_sample.keys())
            temp_views_list = list(new_sample.keys())
            for curr_view in full_views_list:
                #print("t1 ",new_sample[curr_view])
                #print("t1 ",new_sample_proba[curr_view])
                temp_views_list.remove(curr_view)
                for i, curr_class_new in enumerate(new_sample[curr_view]):
                    temp_pop_list = []
                    for j, class_item in enumerate(curr_class_new):
                        #print(class_item)
                        for other_view in temp_views_list:
                            #print("t2 org",new_sample[other_view])
                            #print("t2 org",new_sample_proba[other_view])
                            for l, other_view_class_new in enumerate(new_sample[other_view]):
                                for m, other_class_item in enumerate(other_view_class_new):
                                    # only perform delete if current view contain item, if it's deleted, move on
                                    if new_sample[curr_view][i][j] == new_sample[other_view][l][m]:
                                        if new_sample_proba[curr_view][i][j]>new_sample_proba[other_view][l][m]:
                                            new_sample[other_view][l].pop(m)
                                            new_sample_proba[other_view][l].pop(m)
                                            #print("t2 update",new_sample)
                                            #print("t2 update",new_sample_proba)
                                        else:
                                            temp_pop_list.append(j)
                                        #print(class_item, "check")
                    temp_pop_list.sort(reverse=True)
                    for idx in temp_pop_list:
                        new_sample[curr_view][i].pop(idx)
                        new_sample_proba[curr_view][i].pop(idx)
            #print("apply fix: ",new_sample)
            #print("apply fix: ",new_sample_proba)
            
            for view_idx in range(self.view_num):
                flatten_new_idx[view_idx] = list(itertools.chain(*new_sample["dv"+str(view_idx)]))
                flatten_score[view_idx] = list(itertools.chain(*new_sample_score["dv"+str(view_idx)]))
                flatten_proba[view_idx] = list(itertools.chain(*new_sample_proba["dv"+str(view_idx)]))
                
            #print("check:", flatten_new_idx)
            # if no new label added to L, stop iteration
            if not any(flatten_new_idx.values()):
                self.iterCounter +=1
                print("Remaining unlabel sample is uncertain.")
                break
                
            for view_idx in range(self.view_num):
                self.new_labeled_details["h"+str(view_idx)+"_new_idx"].append(flatten_new_idx[view_idx])
                self.new_labeled_details["h"+str(view_idx)+"_new_score"].append(flatten_score[view_idx])
                self.new_labeled_details["h"+str(view_idx)+"_new_proba"].append(flatten_proba[view_idx])
                
            #print(self.new_labeled_details)
            '''
            Add most confidence samples as new training samples, auto label the samples and remove it from U_prime
            Special Case: if two view classifier give same most confident sample, only 1 datapoint self-labeled
            '''
            roundNewIdx = list(zip(*[value for key, value in new_sample.items()]))
            roundNewScore = list(zip(*[value for key, value in new_sample_score.items()]))
            roundNewProba = list(zip(*[value for key, value in new_sample_proba.items()]))
            roundNew_flatten_unique = []
            for label, round_new in enumerate(zip(roundNewIdx,roundNewScore,roundNewProba)):
                #print(round_new)
                round_new = list(zip(*round_new))
                temp = []
                for index, (idx,score,proba) in enumerate(round_new):
                    for i in range(len(idx)):
                        temp.append((idx[i],score[i],proba[i]))
                #print(temp)
                round_new = set(temp)
                round_new_idx = [item[0] for item in round_new]
                round_new_score = [item[1] for item in round_new]
                round_new_proba = [item[2] for item in round_new]
                self.self_new_all[self.classes_[label]].append(round_new_idx)
                self.self_new_all[self.classes_[label]+"_score"].append(round_new_score)
                self.self_new_all[self.classes_[label]+"_proba"].append(round_new_proba)
                roundNew_flatten_unique.extend(round_new_idx)
                # add label to those new samples
                #print(labels[round_new_idx])
                labels["authorID"][round_new_idx] = self.classes_[label]
                #print(labels[round_new_idx])
                #print(self.classes_[label]," (u' idx): ",round_new_idx)
            #print(roundNew_flatten_unique)
            # extend the labeled sample
            L_plot.extend(roundNew_flatten_unique)
            # remove the labeled sample from U_prime
            U_prime_plot = [x for x in U_prime_plot if x not in roundNew_flatten_unique]
            #print(U_prime_plot)
            # randomly choice view_num*self-label-class-max*class_num examples from u to replenish u_prime
            replenishItem = U_plot[-(int(sum(self.class_ratio))):]
            U_prime_plot.extend(replenishItem)
            U_plot = U_plot[:-len(replenishItem)]
            self.iterCounter +=1
            ''' --------- if reach loop condition k, we extract L from L_plot --------- '''
            if self.iterCounter <= self.k:
                L = copy.deepcopy(L_plot)
                U = copy.deepcopy(U_plot)
                U_prime = copy.deepcopy(U_prime_plot)
                self.self_new_at_k = copy.deepcopy(self.self_new_all)
            
            ''' ----------- if the unlabelled list is running empty -------------- '''
            if not U_prime_plot:
                self.iterCounter +=1
                # ---------------------- test error on validation data --------------------- #
                y_predict = defaultdict(list)
                f1 = defaultdict(list)
                for view_idx in range(self.view_num):
                    y_predict[view_idx]=iter_clf[view_idx].predict(validation_data[view_idx])
                    f1["dv"+str(view_idx)]=f1_score(label_validation,y_predict[view_idx],average='macro')
                    self.f1_on_validation["dv"+str(view_idx)].append(f1["dv"+str(view_idx)])

                self.all_iter_clf.append((self.iterCounter,iter_clf))

                # ---------------- collect the result of different k value ----------------- #
                # --------------- collect result for concatenated features ----------------- #
                temp_train = defaultdict(list)
                temp_clf = defaultdict(list)
                for view_idx in range(self.view_num):
                    temp_train[view_idx] = train_data[view_idx].iloc[L_plot]
                    temp_clf[view_idx] = copy.deepcopy(self.clf[view_idx])
                    temp_clf[view_idx].fit(temp_train[view_idx], labels.iloc[L_plot].values.ravel())

                predict_result = self.combined_predict(temp_clf[0],temp_clf[1],validation_data)
                f1_combined = f1_score(label_validation, predict_result, average='macro')
                self.f1_on_validation_combined.append(f1_combined)
                
                # --------------- print last iteration details ----------- #
                new_train_label = labels.iloc[L_plot]
                print("Iteration",self.iterCounter," h1 new: ", self.new_labeled_details["h0_new_idx"][-1], " score: ", self.new_labeled_details["h0_new_proba"][-1])
                print("Iteration",self.iterCounter," h2 new: ", self.new_labeled_details["h1_new_idx"][-1], " score: ", self.new_labeled_details["h1_new_proba"][-1])
        
        selected_return_iter = self.k
        if self.iterCounter <= self.k:
            selected_return_iter = self.iterCounter
            self.k = self.iterCounter
        print(self.k,"iterations total Labeled number: ", len(L), " Still unlabeled number: ", len(U_prime))
        print(self.iterCounter,"iteration total Labeled number: ", len(L_plot), " Still unlabeled number: ", len(U_prime_plot))
        counter = Counter(L_plot)
        print(counter.most_common(3))
        if len(set(L_plot)) == len(L_plot):
            print("success")
        else:
            print("duplicate found")
            sys.exit("error")
        
        # final train with only k iteration
        final_train = defaultdict(list)
        for view_idx in range(self.view_num):
            final_train[view_idx] = train_data[view_idx].iloc[L]
            self.clf[view_idx].fit(final_train[view_idx], labels.iloc[L].values.ravel())
        ''' Evalutation plot for co-training process, save f1 score vs number of iteration plot '''
        #print(self.f1_on_validation["dv0"])
        #print(self.f1_on_validation["dv1"])
        if (dataset_name != None) and (plot_save_path != None):
            # ----------------- plot h1/h2 result separately -------------------- #
            default_text_based = [self.f1_on_validation["dv0"][0]] * self.iterCounter
            default_citation_based = [self.f1_on_validation["dv1"][0]] * self.iterCounter
            co_train_text_based = self.f1_on_validation["dv0"]
            co_train_citation_based = self.f1_on_validation["dv1"]
            step = np.arange(start=0, stop=self.iterCounter, step=1)
            
            #print("step:",len(step))
            #print("default:",self.iterCounter)
            #print("co-training v1:",len(co_train_text_based))
            #print("co-training v2:",len(co_train_citation_based))
            #print("default concatenate:",self.iterCounter)
            #print("default combined:",self.iterCounter)
            #print("co-training combined:",len(self.f1_on_validation_combined))

            fig = plt.figure(figsize=(8,6))
            ax = plt.axes()
            plt.plot(step, default_text_based, linestyle='dashed', label="Text based default")
            plt.plot(step, default_citation_based, linestyle='dashdot', label="Citation based default")
            plt.plot(step, co_train_text_based, linestyle='solid', marker = "*", label="Text based")
            plt.plot(step, co_train_citation_based, linestyle='dotted', marker = "+", label="Citation based")
            handles, _  = ax.get_legend_handles_labels()
            plt.axvline(x=(selected_return_iter-1), color='r', label="k")
            _, labels = ax.get_legend_handles_labels()
            vertical_line = lines.Line2D([], [],  marker='|', linestyle='None', color="r", markersize=10, markeredgewidth=1.5)
            handles.append(vertical_line)
            ax.autoscale_view()
            legend = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            f1_vs_iteration_plot_name = plot_save_path+dataset_name+"_co_train_iteration_f1(1).png"
            counter = 1
            while os.path.exists(plot_save_path+dataset_name+"_co_train_iteration_f1(%s).png" % counter):
                counter+=1
                f1_vs_iteration_plot_name = plot_save_path+dataset_name+"_co_train_iteration_f1(%s).png" % counter
            plt.savefig(f1_vs_iteration_plot_name, dpi=150, bbox_extra_artists=(legend,), bbox_inches='tight')
            # plt.show()
            plt.close("all")
            
            # -------------------- plot combined result ------------------------- #
            default_concatenate = [self.f1_baseline] * self.iterCounter
            default_combined = [self.f1_on_validation_combined[0]] * self.iterCounter
            co_train_combined = self.f1_on_validation_combined
            fig = plt.figure(figsize=(8,6))
            ax = plt.axes()
            plt.plot(step, default_concatenate, linestyle='dashed', label="Concatenate default")
            plt.plot(step, default_combined, linestyle='dotted', label="Combined default")
            plt.plot(step, co_train_combined, linestyle='solid', marker = "*", label="Combined co-train")
            handles, _  = ax.get_legend_handles_labels()
            plt.axvline(x=(selected_return_iter-1), color='r', label="k")
            _, labels = ax.get_legend_handles_labels()
            vertical_line = lines.Line2D([], [],  marker='|', linestyle='None', color="r", markersize=10, markeredgewidth=1.5)
            handles.append(vertical_line)
            ax.autoscale_view()
            legend = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            f1_vs_iteration_plot_name = plot_save_path+dataset_name+"_co_train_combined_iteration_f1(1).png"
            counter = 1
            while os.path.exists(plot_save_path+dataset_name+"_co_train_combined_iteration_f1(%s).png" % counter):
                counter+=1
                f1_vs_iteration_plot_name = plot_save_path+dataset_name+"_co_train_combined_iteration_f1(%s).png" % counter
            plt.savefig(f1_vs_iteration_plot_name, dpi=150, bbox_extra_artists=(legend,), bbox_inches='tight')
            # plt.show()
            plt.close("all")
            
        return "Training Done"

    def co_train_process_f1(self):
        return self.f1_on_validation["dv0"], self.f1_on_validation["dv1"], self.f1_on_validation_combined, self.f1_baseline

    def get_iter_count(self):
        return self.iterCounter
    
    def get_k(self):
        return self.k

    def predict(self, data):
        y_predict = defaultdict(list)
        for view_idx in range(self.view_num):
            y_predict[view_idx]=self.clf[view_idx].predict(data[view_idx])
        y_predict = [value for key, value in y_predict.items()]
        #fill pred with -1 so we can identify the samples in which error occur
        y_pred = ["-1"] * data[0].shape[0]
        for i, (dv1_y, dv2_y) in enumerate(zip(*y_predict)):
            # if both agree on label
            if dv1_y == dv2_y:
                y_pred[i] = dv1_y
            # If disagree on label and support proba method: We times probability together, choice class have higher probabilities
            elif hasattr(self.clf[0], "predict_proba") and hasattr(self.clf[1], "predict_proba"):
                h1_probas = self.clf[0].predict_proba([data[0].iloc[i]])[0]
                h2_probas = self.clf[1].predict_proba([data[1].iloc[i]])[0]
                final_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(h1_probas, h2_probas)]
                #print("h1 and h2 disagree on",i, " h1 Proba : ",h1_probas, " h2 Proba: ", h2_probas)
                #print("product probas:",final_y_probas)
                max_prob_idx = final_y_probas.index(max(final_y_probas))
                y_pred[i] = self.classes_[max_prob_idx]
                #print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            # If disagree on label and not support proba method: We use decision_function to first calculate probability,
            # then we times calculate probability together, choice class have higher probabilities
            elif hasattr(self.clf[0], "decision_function") and hasattr(self.clf[1], "decision_function"):
                dv1_distance = self.clf[0].decision_function([data[0].iloc[i]])
                dv2_distance = self.clf[1].decision_function([data[1].iloc[i]])
                if len(self.clf1.classes_)==2:
                    h1_probas = np.array([1-self.sigmoid(dv1_distance), self.sigmoid(dv1_distance)])
                    h2_probas = np.array([1-self.sigmoid(dv2_distance), self.sigmoid(dv2_distance)])
                else:
                    h1_probas = self.softmax(dv1_distance)
                    h2_probas = self.softmax(dv2_distance)
                final_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(h1_probas, h2_probas)]
                #print("h1 and h2 disagree on",i, " h1 Proba : ",h1_probas, " h2 Proba: ", h2_probas)
                #print("product probas:",final_y_probas)
                max_prob_idx = final_y_probas.index(max(final_y_probas))
                y_pred[i] = self.classes_[max_prob_idx]
                #print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            # disagree and not support any confidence measure
            else:
                sys.exit("Error occur")

        # convert final result to np array
        y_pred_np_array = np.asarray(y_pred)
        return y_pred_np_array

    def predict_proba(self, dv1, dv2):
        # the predicted probabilities is simply a product (*) of probabilities given from each classifier trained
        h1_probas = self.clf1.predict_proba(dv1)
        h2_probas = self.clf2.predict_proba(dv2)
        
        proba = (h1_probas*h2_probas)
        return proba
    
    ''' ------------- function for optimize parameter k only ---------------- '''
    def combined_predict(self, dv1_clf, dv2_clf, data):
        dv1=data[0]
        dv2=data[1]
        dv1_predict = dv1_clf.predict(dv1)
        dv2_predict = dv2_clf.predict(dv2)
        #fill pred with -1 so we can identify the samples in which error occur
        y_pred = ["-1"] * dv1.shape[0]
        for i, (dv1_y, dv2_y) in enumerate(zip(dv1_predict, dv2_predict)):
            # if both agree on label
            if dv1_y == dv2_y:
                y_pred[i] = dv1_y
            # If disagree on label and support proba method: We times probability together, choice class have higher probabilities
            elif hasattr(dv1_clf, "predict_proba") and hasattr(dv2_clf, "predict_proba"):
                h1_probas = dv1_clf.predict_proba([dv1.iloc[i]])[0]
                h2_probas = dv2_clf.predict_proba([dv2.iloc[i]])[0]
                final_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(h1_probas, h2_probas)]
                #print("h1 and h2 disagree on",i, " h1 Proba : ",h1_probas, " h2 Proba: ", h2_probas)
                #print("product probas:",final_y_probas)
                max_prob_idx = final_y_probas.index(max(final_y_probas))
                y_pred[i] = self.classes_[max_prob_idx]
                #print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            # If disagree on label and not support proba method: We use decision_function to first calculate probability,
            # then we times calculate probability together, choice class have higher probabilities
            elif hasattr(dv1_clf, "decision_function") and hasattr(dv2_clf, "decision_function"):
                dv1_distance = dv1_clf.decision_function([dv1.iloc[i]])
                dv2_distance = dv2_clf.decision_function([dv2.iloc[i]])
                if len(dv1_clf.classes_)==2:
                    h1_probas = np.array([1-self.sigmoid(dv1_distance), self.sigmoid(dv1_distance)])
                    h2_probas = np.array([1-self.sigmoid(dv2_distance), self.sigmoid(dv2_distance)])
                else:
                    h1_probas = self.softmax(dv1_distance)
                    h2_probas = self.softmax(dv2_distance)
                final_y_probas = [proba_y1 * proba_y2 for (proba_y1, proba_y2) in zip(h1_probas, h2_probas)]
                #print("h1 and h2 disagree on",i, " h1 Proba : ",h1_probas, " h2 Proba: ", h2_probas)
                #print("product probas:",final_y_probas)
                max_prob_idx = final_y_probas.index(max(final_y_probas))
                y_pred[i] = self.classes_[max_prob_idx]
                #print("result idx: ", max_prob_idx, " result: ",y_pred[i])
            # disagree and not support any confidence measure
            else:
                sys.exit("Error occur")

        # convert final result to np array
        y_pred_np_array = np.asarray(y_pred)
        return y_pred_np_array


In [10]:
def simulate_co_training_situation(all_train_label,init_labeled_size,init_validation_size):
    # ----------- set some labeled data as unlabeled ------------ #
    # 1. obtain data ratio
    c = Counter(all_train_label)
    data_ratio = [(i, c[i] / len(all_train_label)) for i in c]
    print(data_ratio)
    # 2. calculate per class size 
    # co_train_per_class_size contain (label,initial train size for each class, initial validation data for each class)
    co_train_per_class_size = [(label, round(ratio*init_labeled_size),round(ratio*init_validation_size)) for label, ratio in data_ratio]
    print(co_train_per_class_size)
    # 3.Initialize train and validation sample index list save it for later use
    temp_train_label = all_train_label.copy()
    label_train_sample_idx = []
    validation_sample_idx = []
    # 4. random draw both train labeled samples and validation samples, mark other as unlabeled
    # we could also use validation samples to improve performance
    for unique_label, training_size, validation_size in co_train_per_class_size:
        curr_label_idx = [idx for (idx, label) in temp_train_label.iteritems() if label == unique_label]
        curr_label_size = len(curr_label_idx)
        # 1. get train sample idx
        keep_as_labelled_train_p1 = random.sample(curr_label_idx, training_size)
        curr_label_idx_no_train_p2 = [x for x in curr_label_idx if x not in keep_as_labelled_train_p1]
        label_train_sample_idx += keep_as_labelled_train_p1
        # 2. get validation sample idx
        temp_validation_sample_idx = random.sample(curr_label_idx_no_train_p2, validation_size)
        validation_sample_idx += temp_validation_sample_idx
        unlabel_item_idx = [x for x in curr_label_idx_no_train_p2 if x not in temp_validation_sample_idx]
        # 3. set other samples to -1 as unlabeled
        for unlabel_idx in unlabel_item_idx:
            temp_train_label[unlabel_idx]=-1
    return label_train_sample_idx, temp_train_label, validation_sample_idx

In [11]:
import copy
import random
import collections

import seaborn as sns

from statistics import mean 
from collections import Counter
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score,f1_score,accuracy_score

# cross validation
def k_fold_cv_all_algorithm(dv1, dv2, label, init_labeled_size, muti_view_clf=[], combined_clf=[],
                            num_fold=10, dataset_name=None, plot_save_path=None, validation=False):
    # set validation dataset as 10% of all data
    if validation:
        init_validation_size = len(label)*0.1
    else:
        init_validation_size = 0
    kf = StratifiedKFold(n_splits=num_fold)
    allTrueLabel = []
    co_train_algorithm = [name for clf,name in muti_view_clf]
    baseline_algorithm = [name for clf,name in combined_clf]
    allPredLabel = collections.defaultdict(list)
    all_fold_co_train_iter_count = collections.defaultdict(list)
    all_fold_co_train_iter_result = collections.defaultdict(list)
    
    all_fold_statistic = []
    fold = 0
    # convert different input type to dataframe for consistency
    dv1 = pd.DataFrame(dv1)
    dv2 = pd.DataFrame(dv2)
    
    for train_index, test_index in kf.split(dv1, label):
        fold +=1
        # print("TRAIN:", train_index, " \n TEST:", test_index)
        # ---------------split train and test -------------------- #
        dv1_train, dv1_test = dv1.iloc[train_index], dv1.iloc[test_index]
        dv2_train, dv2_test = dv2.iloc[train_index], dv2.iloc[test_index]
        all_train_label, test_label = label.iloc[train_index], label.iloc[test_index]
        '''
        Plot true labeled result for different view on each fold, use PCA to reduce views to 2d
        Real training process will not using PCA to reduce it's dimension 
        '''
        pca = PCA(n_components=2)
        ''' 
        Comparison Part 1: Use two view concatenated features and train with supervise learning (up bound)
        Notice we train supervise learning with all labeled data, no data has been mark as unlabeled
        '''
        # ---------- collect per fold statistic ---------------------- #
        curr_fold_statistic = {'author': dataset_name, 'fold':fold, 'test_size': dv1_test.shape[0]} 
        ub_concatenated_train = pd.concat([dv1_train,dv2_train], axis=1, ignore_index=True)
        concatenated_test = pd.concat([dv1_test,dv2_test], axis=1, ignore_index=True)
        for in_clf, clf_name in combined_clf:
            UB_clf = copy.deepcopy(in_clf) 
            UB_clf.fit(ub_concatenated_train, all_train_label)
            ub_pred = UB_clf.predict(concatenated_test)
            print(clf_name+"-UB f1: ", metrics.classification_report(test_label, ub_pred))
            print(metrics.confusion_matrix(test_label, ub_pred).ravel())
            curr_fold_statistic[clf_name+"-UB f1"] = f1_score(test_label.values.tolist(), ub_pred,average='macro')
            allPredLabel[clf_name+"-UB predict label"].append(ub_pred)

        ''' 
        ############# Notice, part 2 and part 3 will run 10 times to obtain an average (stable result) ###############
        Comparison Part 2: Use two view and train with co-training (Check effective of co-training)
        Notice here we are simulating situation for co-training by set most of data to unlabelled
        '''
        curr_fold_statistic["train_size"] = init_labeled_size
        curr_fold_statistic['validation_size']=init_validation_size
        curr_fold_statistic["unlabel_size"] = len(all_train_label) - init_labeled_size
        # ------------ collect statistic for all runs ---------------- #
        curr_fold_all_runs = collections.defaultdict(list)
        curr_fold_co_train_iter_count = collections.defaultdict(list)
        curr_fold_co_train_iter_result = collections.defaultdict(list)
        for i in range(20):
            label_train_sample_idx, final_train_label, validation_sample_idx = simulate_co_training_situation(all_train_label,init_labeled_size,init_validation_size)
            print(label_train_sample_idx)
            # -------------------- train with co-training ------------------- #
            for in_clf, clf_name in muti_view_clf: 
                if plot_save_path == None:
                    print("plot off")
                    per_clf_plot_save_path = None
                else:
                    per_clf_plot_save_path = plot_save_path+dataset_name+"/fold"+str(fold)+"/"+clf_name+"/"
                    if not os.path.exists(per_clf_plot_save_path):
                        os.makedirs(per_clf_plot_save_path)
                co_train_clf = copy.deepcopy(in_clf)
                co_train_clf.set_PCA(pca)
                co_train_clf.fit(train_data=[dv1_train.copy(), dv2_train.copy(), final_train_label.copy()], 
                                 validation_data=[dv1_test, dv2_test, test_label],
                                 dataset_name=dataset_name, plot_save_path=per_clf_plot_save_path)
                curr_fold_all_runs[clf_name+"_total_iteration"].append(co_train_clf.get_iter_count())
                curr_fold_all_runs[clf_name+"_select_iteration"].append(co_train_clf.get_k())
                # -------------- get self-labeled sample index --------------- #
                self_new_all, self_new_k_iter = co_train_clf.get_self_labeled_sample()
                #print("Self labeled at iter k sample idx: ", self_new_k_iter)
                #print("Self labeled idx: ", self_new_all)
                
                self_new_all_idx = [idx for idx in self_new_all.values()]
                all_self_new_idx = list(set([val for sublist in self_new_all_idx for subsublist in sublist for val in subsublist]))
                curr_fold_all_runs[clf_name+'_total_self_labeled'].append(len(all_self_new_idx))
                
                self_new_k_iter_idx = [idx for idx in self_new_k_iter.values()]
                iter_k_self_new_idx = list(set([val for sublist in self_new_k_iter_idx for subsublist in sublist for val in subsublist]))
                curr_fold_all_runs[clf_name+'_iter_k_self_labeled'].append(len(iter_k_self_new_idx))
                #print("Self labeled idx: ", all_self_new_idx, " Size: ", len(all_self_new_idx))
                #print("Self labeled at iter k sample idx: ", iter_k_self_new_idx, " Size: ", len(iter_k_self_new_idx))
                # ------------- get predicted label for test set ------------- #
                co_train_predict = co_train_clf.predict([dv1_test, dv2_test])
                print(clf_name+" f1: ", metrics.classification_report(test_label, co_train_predict))
                print(metrics.confusion_matrix(test_label, co_train_predict).ravel())
                curr_fold_all_runs[clf_name+" f1"].append(f1_score(test_label.values.tolist(), co_train_predict,average='macro'))
                allPredLabel[clf_name+" predict label"].append(co_train_predict)
                # ------------- get co-training per iterations f1 score for plot  ---------- #
                curr_fold_co_train_iter_count[clf_name].append(co_train_clf.get_iter_count())
                # --------------- h1/h2 plot ----------------- #
                diff_iter_f1_dv1, diff_iter_f1_dv2, diff_iter_f1_combined, concatenate_f1 = co_train_clf.co_train_process_f1()
                curr_fold_co_train_iter_result[clf_name+"_dv1"].append(diff_iter_f1_dv1)
                curr_fold_co_train_iter_result[clf_name+"_dv2"].append(diff_iter_f1_dv2)
                # ------------- combined plot ---------------- #
                curr_fold_co_train_iter_result[clf_name+"_combined"].append(diff_iter_f1_combined)
                curr_fold_co_train_iter_result[clf_name+"_concatenate_baseline"].append(concatenate_f1)

            ''' 
            Comparison Part 3: Use two view concatenated features and train with supervise learning (lower bound)
            Notice we only train supervise learning with labeled train, data mark as unlabeled is not used
            '''
            lb_concatenated_train = pd.concat([dv1_train.loc[label_train_sample_idx],
                                               dv2_train.loc[label_train_sample_idx]],axis=1, ignore_index=True)
            lb_train_label = [final_train_label[idx] for idx in label_train_sample_idx]
            for in_clf, clf_name in combined_clf:
                LB_clf = copy.deepcopy(in_clf)
                LB_clf.fit(lb_concatenated_train, lb_train_label)
                lb_pred = LB_clf.predict(concatenated_test)
                print(clf_name+"-LB f1: ", metrics.classification_report(test_label, lb_pred))
                print(metrics.confusion_matrix(test_label, lb_pred).ravel())
                curr_fold_all_runs[clf_name+"-LB f1"].append(f1_score(test_label.values.tolist(), lb_pred,average='macro'))
                allPredLabel[clf_name+"-LB predict label"].append(lb_pred)

        # ---------- calculate average of all runs to obtain a stable result ------------- #
        for key,value in curr_fold_all_runs.items():
            #print(key,":",value,":",value,":",mean(value))
            curr_fold_statistic[key] = mean(value)
        allTrueLabel.extend(test_label.values.tolist())
        all_fold_statistic.append(curr_fold_statistic)
        
        # -------- plot averaged result for 20 runs in each fold ------------- #
        for clf_name in co_train_algorithm:
            # each run have different iteration, find minimum of iteration in all runs.
            min_iter_in_runs = min(curr_fold_co_train_iter_count[clf_name])
            print(clf_name," per run iteration: ", curr_fold_co_train_iter_count[clf_name], "min iter is: ", min_iter_in_runs)
            for idx, sublist in enumerate(curr_fold_co_train_iter_result[clf_name+"_dv1"]):
                curr_fold_co_train_iter_result[clf_name+"_dv1"][idx] = sublist[:min_iter_in_runs]
            for idx, sublist in enumerate(curr_fold_co_train_iter_result[clf_name+"_dv2"]):
                curr_fold_co_train_iter_result[clf_name+"_dv2"][idx] = sublist[:min_iter_in_runs]
            for idx, sublist in enumerate(curr_fold_co_train_iter_result[clf_name+"_combined"]):
                curr_fold_co_train_iter_result[clf_name+"_combined"][idx] = sublist[:min_iter_in_runs]
            # -------- mean with respect to all run --------- #
            mean_curr_fold_diff_iter_dv1 = np.mean(curr_fold_co_train_iter_result[clf_name+"_dv1"], axis=0)
            mean_curr_fold_diff_iter_dv2 = np.mean(curr_fold_co_train_iter_result[clf_name+"_dv2"], axis=0)
            mean_curr_fold_diff_iter_combined = np.mean(curr_fold_co_train_iter_result[clf_name+"_combined"], axis=0)
            mean_curr_fold_diff_iter_concatenate_baseline = np.mean(curr_fold_co_train_iter_result[clf_name+"_concatenate_baseline"])
            # -------- initial variables for plot ------------ #
            default_text_based = [mean_curr_fold_diff_iter_dv1[0]] * min_iter_in_runs
            default_citation_based = [mean_curr_fold_diff_iter_dv2[0]] * min_iter_in_runs
            co_train_text_based = mean_curr_fold_diff_iter_dv1
            co_train_citation_based = mean_curr_fold_diff_iter_dv2
            step = np.arange(start=0, stop=min_iter_in_runs, step=1)
            
            # ----------- plot h1/h2 details ----------------------- #
            fig = plt.figure(figsize=(8,6))
            ax = plt.axes()
            plt.plot(step, default_text_based, linestyle='dashed', label="Text based default")
            plt.plot(step, default_citation_based, linestyle='dashdot', label="Citation based default")
            plt.plot(step, co_train_text_based, linestyle='solid', marker = "*", label="Text based")
            plt.plot(step, co_train_citation_based, linestyle='dotted', marker = "+", label="Citation based")
            handles, _  = ax.get_legend_handles_labels()
            plt.axvline(x=(min(min_iter_in_runs-1,30)), color='r', label="k")
            _, labels = ax.get_legend_handles_labels()
            vertical_line = lines.Line2D([], [],  marker='|', linestyle='None', color="r", markersize=10, markeredgewidth=1.5)
            handles.append(vertical_line)
            ax.autoscale_view()
            legend = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            figure_name = plot_save_path+dataset_name+"/fold"+str(fold)+"/"+clf_name+"_mean_diff_iter_f1.png"
            plt.savefig(fname=figure_name, dpi=150, bbox_extra_artists=(legend,), bbox_inches='tight')
            #plt.show()
            plt.close("all")
            # ---------------- plot combined details ------------------ #
            default_concatenated = [mean_curr_fold_diff_iter_concatenate_baseline] * min_iter_in_runs
            default_combined = [mean_curr_fold_diff_iter_combined[0]] * min_iter_in_runs
            co_train_combined = mean_curr_fold_diff_iter_combined
            fig = plt.figure(figsize=(8,6))
            ax = plt.axes()
            plt.plot(step, default_concatenated, linestyle='dashed', label="Concatenate default")
            plt.plot(step, default_combined, linestyle='dotted', label="Combined default")
            plt.plot(step, co_train_combined, linestyle='solid', marker = "+", label="Combined Co-train")
            handles, _  = ax.get_legend_handles_labels()
            plt.axvline(x=(min(min_iter_in_runs-1,30)), color='r', label="k")
            _, labels = ax.get_legend_handles_labels()
            vertical_line = lines.Line2D([], [],  marker='|', linestyle='None', color="r", markersize=10, markeredgewidth=1.5)
            handles.append(vertical_line)
            ax.autoscale_view()
            legend = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            figure_name = plot_save_path+dataset_name+"/fold"+str(fold)+"/"+clf_name+"_mean_combined_diff_iter_f1.png"
            plt.savefig(fname=figure_name, dpi=150, bbox_extra_artists=(legend,), bbox_inches='tight')
            #plt.show()
            plt.close("all")
            # ---------- collect everyfold average statistic for all fold average ----------- #
            all_fold_co_train_iter_count[clf_name].append(min_iter_in_runs)
            all_fold_co_train_iter_result[clf_name+"_dv1"].append(mean_curr_fold_diff_iter_dv1)
            all_fold_co_train_iter_result[clf_name+"_dv2"].append(mean_curr_fold_diff_iter_dv2)
            all_fold_co_train_iter_result[clf_name+"_combined"].append(mean_curr_fold_diff_iter_combined)
            all_fold_co_train_iter_result[clf_name+"_concatenated_baseline"].append(mean_curr_fold_diff_iter_concatenate_baseline)
    
    ''' # --------------- plot per fold result f1 variance --------------- # '''
    if (plot_save_path !=None) and (dataset_name != None):
        all_per_fold_f1_score_variance_plot = pd.DataFrame(all_fold_statistic)
        #print(all_per_fold_f1_score_variance_plot)
        sns.set(rc={'figure.figsize':(10,8)})
        plot_temp_f1 = pd.DataFrame()
        for clf_name in co_train_algorithm:
            clf_temp_f1 = all_per_fold_f1_score_variance_plot[[clf_name+' f1']].values
            plot_temp_f1[clf_name]=clf_temp_f1.flatten()
        for clf_name in baseline_algorithm:
            for clf_type in ["-LB"]:
                clf_temp_f1 = all_per_fold_f1_score_variance_plot[[clf_name+clf_type+' f1']].values
                plot_temp_f1[clf_name]=clf_temp_f1.flatten()
        plot_temp_f1 = pd.melt(plot_temp_f1, var_name='methods', value_name='f1')
        #print(plot_temp_f1)
        ax = sns.boxplot(x="methods", y="f1", data=plot_temp_f1)
        ax = sns.swarmplot(x="methods", y="f1", data=plot_temp_f1, color=".25")
        ax.set_title(dataset_name+" result variance within "+str(num_fold)+" fold")
        plt.savefig(plot_save_path+dataset_name+"/all_method_result_variance.png", dpi=150)
        #plt.show()
        plt.close("all")

        '''  plot averaged f1 score wrt different fold in different iterations in co-training process '''
        for clf_name in co_train_algorithm:
            # each fold have different iteration, find minimum of iteration and 
            min_iter_in_folds = min(all_fold_co_train_iter_count[clf_name])
            print(clf_name," per fold iteration: ", all_fold_co_train_iter_count[clf_name]," min iter: ",min_iter_in_folds)
            for idx, sublist in enumerate(all_fold_co_train_iter_result[clf_name+"_dv1"]):
                all_fold_co_train_iter_result[clf_name+"_dv1"][idx] = sublist[:min_iter_in_folds]
            for idx, sublist in enumerate(all_fold_co_train_iter_result[clf_name+"_dv2"]):
                all_fold_co_train_iter_result[clf_name+"_dv2"][idx] = sublist[:min_iter_in_folds]
            for idx, sublist in enumerate(all_fold_co_train_iter_result[clf_name+"_combined"]):
                all_fold_co_train_iter_result[clf_name+"_combined"][idx] = sublist[:min_iter_in_folds]
            
            # -------- mean with respect to all fold --------- #
            mean_all_fold_diff_iter_dv1 = np.mean(all_fold_co_train_iter_result[clf_name+"_dv1"], axis=0)
            mean_all_fold_diff_iter_dv2 = np.mean(all_fold_co_train_iter_result[clf_name+"_dv2"], axis=0)
            mean_all_fold_diff_iter_combined = np.mean(all_fold_co_train_iter_result[clf_name+"_combined"], axis=0)
            mean_all_fold_diff_iter_baseline = np.mean(all_fold_co_train_iter_result[clf_name+"_concatenated_baseline"], axis=0)
            # -------- initial variables for plot ------------ #
            step = np.arange(start=0, stop=min_iter_in_folds, step=1)
            default_text_based = [mean_all_fold_diff_iter_dv1[0]] * min_iter_in_folds
            default_citation_based = [mean_all_fold_diff_iter_dv2[0]] * min_iter_in_folds
            co_train_text_based = mean_all_fold_diff_iter_dv1
            co_train_citation_based = mean_all_fold_diff_iter_dv2
            # --------------------- plot averaged training details ----------------------- #
            # -------------- h1/h2 ---------------- #
            fig = plt.figure(figsize=(8,6))
            ax = plt.axes()
            plt.plot(step, default_text_based, linestyle='dashed', label="Text based default")
            plt.plot(step, default_citation_based, linestyle='dashdot', label="Citation based default")
            plt.plot(step, co_train_text_based, linestyle='solid', marker = "*", label="Text based")
            plt.plot(step, co_train_citation_based, linestyle='dotted', marker = "+", label="Citation based")
            handles, _  = ax.get_legend_handles_labels()
            plt.axvline(x=min(min_iter_in_folds-1,30), color='r', label="k")
            _, labels = ax.get_legend_handles_labels()
            vertical_line = lines.Line2D([], [],  marker='|', linestyle='None', color="r", markersize=10, markeredgewidth=1.5)
            handles.append(vertical_line)
            ax.autoscale_view()
            legend = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            plt.savefig((plot_save_path+dataset_name+"/"+clf_name+"_mean_diff_iter_f1.png"), dpi=150, bbox_extra_artists=(legend,), bbox_inches='tight')
            #plt.show()
            plt.close("all")
            # ----------- combined ------------- #
            default_concatenated = [mean_all_fold_diff_iter_baseline] * min_iter_in_folds
            default_combined = [mean_all_fold_diff_iter_combined[0]] * min_iter_in_folds
            co_train_combined = mean_all_fold_diff_iter_combined
            fig = plt.figure(figsize=(8,6))
            ax = plt.axes()
            plt.plot(step, default_concatenated, linestyle='dashed', label="Concatenate default")
            plt.plot(step, default_combined, linestyle='dotted', label="Combined default")
            plt.plot(step, co_train_combined, linestyle='solid', marker = "+", label="Combined Co-train")
            handles, _  = ax.get_legend_handles_labels()
            plt.axvline(x=min(min_iter_in_folds-1,30), color='r', label="k")
            _, labels = ax.get_legend_handles_labels()
            vertical_line = lines.Line2D([], [],  marker='|', linestyle='None', color="r", markersize=10, markeredgewidth=1.5)
            handles.append(vertical_line)
            ax.autoscale_view()
            legend = ax.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
            plt.xlabel('Co-Training Iterations')
            plt.ylabel('F1 score')
            figure_name = plot_save_path+dataset_name+"/"+clf_name+"_mean_combined_diff_iter_f1.png"
            plt.savefig(fname=figure_name, dpi=150, bbox_extra_artists=(legend,), bbox_inches='tight')
            #plt.show()
            plt.close("all")
    
    ''' The results of a k-fold cross-validation run are often summarized with the mean of the model scores.'''
    final_f1_score = []
    for clf_name in co_train_algorithm:
        clf_all_fold_f1 =[]
        for per_fold_statistic in all_fold_statistic:
            clf_per_fold_f1 = per_fold_statistic[clf_name+" f1"]
            clf_all_fold_f1.append(clf_per_fold_f1)
        clf_mean_f1 = np.mean(clf_all_fold_f1, axis=0)
        final_f1_score.append((clf_name,clf_mean_f1))
    for clf_name in baseline_algorithm:
        for clf_type in ["-UB","-LB"]:
            clf_all_fold_f1 =[]
            for per_fold_statistic in all_fold_statistic:
                clf_per_fold_f1 = per_fold_statistic[clf_name+clf_type+" f1"]
                clf_all_fold_f1.append(clf_per_fold_f1)
            clf_mean_f1 = np.mean(clf_all_fold_f1, axis=0)
            final_f1_score.append((clf_name+clf_type,clf_mean_f1))
    
    return final_f1_score, all_fold_statistic


In [14]:
import numpy as np
import collections
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score
from sklearn.decomposition import PCA

# fix random seed for reproducibility
np.random.seed(1)

# loop through all files in directory add name to name list
fileDir = "../Data/"+Dataset+"/canopies_labeled/"
listfiles = os.listdir(fileDir)

init_labeled_size = 10

co_lr_diff_embedding_final_result = collections.defaultdict(list)

#---------------- load different embedding combination ---------------#
for v1_emb, v2_emb in zip(pp_text, pp_citation):
    # read embeddings
    print("Load text embedding: ", v1_emb)
    viewone_text_emb = com_func.read_text_embedding(emb_type=v1_emb, training_size = "140k")
    print("Load citation embedding: ", v2_emb)
    viewtwo_citation_embedding = com_func.read_citation_embedding_sorted(emb_type = v2_emb, labeled_only = True)
    # print(viewone_text_emb[0])
    # print(viewtwo_citation_embedding[0])
    
    threshold_change_all_method_f1s = collections.defaultdict(list)
    # -------------- different threshold (step by 10) -----------------------#
    for step_threshold in range(threshold_lower, threshold_upper, 10):
        plot_save_path = "../../plot/3_co_train_detail_plots/threshold="+str(step_threshold)+"/temp_cs=0.5/(Fixed class ratio)10RPFA_V1="+v1_emb+"_V2="+v2_emb+"/"
        #plot_save_path = None
        threshold_change_all_method_f1s["threshold"].append(step_threshold)
        # collect statistic to output
        statistic_detail = collections.defaultdict(list)
        total_selected_group = 0
        selected_binary_case_group = 0

        # ------- different name group in all name group --------------------#
        for file in listfiles:
            # group name
            temp = file.split("_")
            name = str(temp[1]+"_"+temp[-1])
            print("For name: ",name)
            # read label (pid : author ORCID) from file
            data = com_func.read_pid_aid(fileDir+file)
            labeled_mask = data["authorID"] != "-1"
            labeled_data = data[labeled_mask]
            print(labeled_data.shape)
            # ---------------- collect all labeled sample -------------------- #
            # ---------------- if use all samples as negative --------------- #
            all_labeled_samples = labeled_data["paperID"].tolist()
            authorCounter = com_func.select_productive_groups(labeled_data=labeled_data, 
                                                              threshold=threshold_select_name_group)
            ''' 
            Case 1: no author under this name have written more than threshold number of papers, dataset not used
            Case 2: only one author under this name written more than threshold number of papers, dataset not used
            Case 3: 2 or more author under this name written more than threshold number of papers, dataset used
            '''
            if(len(authorCounter)==0) or (len(authorCounter)==1):
                print(name," pass")
            else:
                total_selected_group+= 1
                '''
                Case 1: All papers under name group is used. We include authors written less than threshold of paper,
                        and treat it as negative class. This will be OVR.(Not used)
                Case 2: All papers under name group is used. We include authors written less than threshold of paper,
                        and perform muti-class classification (Not used)
                Case 3: Only Include author with more than threshold number of paper and perform muti-class classification (Not used)
                Case 5: Only Include author with more than threshold number of paper and perform OVR(used)
                Case 4: Only Include author with more than threshold number of paper and only select binary case(used)
                '''
                if apply_threshold_to_name_group_samples == True:
                    # ---------- only use sample pass threshold ------- #
                    #-------- only select authors in name group are very productive (more than threshold)---------#
                    labeled_data, author_list, _= com_func.only_select_productive_authors(labeled_data, step_threshold)
                    # ----------------- if use filtered samples as negative  --------- #
                    filtered_all_labeled_samples = labeled_data["paperID"].tolist()
                else:
                    # ----------- use all sample in name group --------- #
                    author_list = com_func.productive_authors_list(labeled_data, step_threshold)
                    print(name, " name group sample size: ",labeled_data.shape)
                # -------------- extract all samples for name group -------------- #
                # for each name group
                # read in labeled data
                labeled_viewone_text = com_func.extract_sorted_embedding(viewone_text_emb, labeled_data["paperID"])
                print(labeled_viewone_text.shape)
                labeled_viewtwo_citation = com_func.extract_sorted_embedding(viewtwo_citation_embedding, labeled_data["paperID"])
                print(labeled_viewtwo_citation.shape)
                # ---------------- shuffle the data ----------------- #
                labeled_data = labeled_data.sample(frac=1).reset_index(drop=True)
                ''' Alignment and deal with missing data rows: 
                Case 1: fill missing data with 0, using labeled_viewtwo_citation.fillna(0, inplace=True)
                Case 2: fill missing data with average of other data
                Case 3: drop the sample contains missing data
                '''
                labeled_viewone_text = pd.merge(labeled_data, labeled_viewone_text, left_on="paperID", right_on = [0], how = "left")
                labeled_viewtwo_citation = pd.merge(labeled_data, labeled_viewtwo_citation, left_on="paperID", right_on = [0], how = "left")
                rows_with_nan = [index for index, row in labeled_viewtwo_citation.iterrows() if row.isnull().any()]
                print("Papers with no citation: ", rows_with_nan)
                print("Total paper with citation: ", len(labeled_viewone_text)-len(rows_with_nan))
                labeled_viewtwo_citation.fillna(0, inplace=True)
                #rows_with_nan = [index for index, row in labeled_viewtwo_citation.iterrows() if row.isnull().any()]
                #print(rows_with_nan)
                #labeled_viewone_text = labeled_viewone_text.drop(rows_with_nan).reset_index(drop=True)
                #labeled_viewtwo_citation = labeled_viewtwo_citation.drop(rows_with_nan).reset_index(drop=True)
                unique_labels = labeled_viewone_text.authorID.unique()
                map_dict = {}
                for idx, unique_label in enumerate(unique_labels):
                    map_dict[unique_label] = name+"_"+str(idx+1)
                true_label = labeled_viewone_text["authorID"].replace(map_dict)
                
                '''
                only work on binary case, ignored multi-class case
                We need to check whether the name group only contain binary case or not
                '''
                if len(author_list)==2:
                    #if name in ["p_robinson","r_reis", "a_silva","w_lee"]:
                    #   print(name, " Pass for error checking")
                    #    continue
                    selected_binary_case_group +=1
                    print(name + " is binary case")
                    viewone_text_final = labeled_viewone_text.drop(["paperID", "authorID", 0], axis=1)
                    viewtwo_citation_final = labeled_viewtwo_citation.drop(["paperID", "authorID", 0], axis=1)
                    print(viewone_text_final.shape)
                    print(viewtwo_citation_final.shape)
                    ''' Apply different algorithm:
                    Part 1: Basic supervised algorithm 
                    Part 2: Basic co-training algorithm 
                    Part 3: 2 clf co-training 
                    Part 4: Improved co-training algorithm (Self-proposed with all different improvement)
                    '''
                    # -------------------- part 1 ------------------------- #
                    LR_clf = LogisticRegression(solver= "liblinear")
                    SVM_clf = SVC(gamma="auto", kernel='linear')
                    # -------------------- part 2 ------------------------- #
                    initial_cotrain_parameters = {"k":30}
                    co_LR_clf = Co_training_clf(clf1=LogisticRegression(solver= "liblinear"),**initial_cotrain_parameters)
                    ''' For co-training with SVM
                    Case 1: Using Scikit-learn where we set probability=True
                    Case 1 details: Scikit-learn uses LibSVM internally, and this in turn uses Platt scaling
                    Platt scaling requires first training the SVM as usual, then optimizing parameter 
                    vectors A and B such that: P(y|X) = 1 / (1 + exp(A * f(X) + B))
                    where f(X) is the signed distance of a sample from the hyperplane
                    (scikit-learn's decision_function method).
                    Case 2: Using decision_function to get signed distance of sample from the hyperplane, calculate proba
                    Case 2 details: use sigmoid (binary)/ softmax (muti-class) for calculate probability based on 
                    signed distance of sample from the hyperplane. Then follow same step as case 1. The result is relatively
                    bad
                    '''
                    # ----------- Case 1 Using Scikit-learn where we set probability=True ------ #
                    co_SVM_clf = Co_training_clf(clf1=SVC(gamma="auto", kernel='linear',probability=True),**initial_cotrain_parameters)
                    # ------- Case 2 Using decision_function get distance, calculate proba------ #
                    #co_svm_clf = Co_training_clf(clf1=SVC(gamma="auto", kernel='linear'),**initial_cotrain_parameters)
                    #co_train_clfs = [(co_svm_clf,"co_train_SVM")]
                    # -------------------- part 3 ------------------------- #
                    # two different clf with basic co-training
                    co_LR_SVM_clf = Co_training_clf(clf1 = LogisticRegression(solver= "liblinear"),
                                                    clf2 = SVC(gamma="auto", kernel='linear',probability=True),
                                                    **initial_cotrain_parameters)
                    # -------------------- part 4 ------------------------- #
                    improved_cotrain_parameters = {"view_num":2,"k":30}
                    improved_co_LR = Improved_co_training_clf(clf = [LogisticRegression(solver= "liblinear")], 
                                                              **improved_cotrain_parameters)
                    # -------------------- train together ----------------- #
                    baseline_clfs = [(LR_clf,"LR"),(SVM_clf,"SVM")]
                    final_co_train_clfs = [(co_LR_clf,"co_LR"), (co_SVM_clf,"co_SVM"),
                                           (co_LR_SVM_clf, "co_LR_SVM"), (improved_co_LR, "Improved_co_LR")]
                    #baseline_clfs = [(LR_clf,"LR")]
                    #final_co_train_clfs = [(co_LR_clf,"co_LR"),(improved_co_LR, "Improved_co_LR")]
                    final_f1_score, cv_per_fold_status= k_fold_cv_all_algorithm(dv1=viewone_text_final,
                                                                                dv2=viewtwo_citation_final,
                                                                                label=true_label,
                                                                                init_labeled_size=init_labeled_size,
                                                                                muti_view_clf=final_co_train_clfs,
                                                                                combined_clf=baseline_clfs,
                                                                                num_fold=5,
                                                                                dataset_name=name,
                                                                                plot_save_path=plot_save_path)
                    
                    statistic_detail['Name'].append(name)
                    statistic_detail['Total_sample_size'].append(len(true_label))
                    statistic_detail['Train_size'].append(cv_per_fold_status[0]["train_size"])
                    statistic_detail['Test_size'].append(cv_per_fold_status[0]["test_size"])
                    statistic_detail["All_fold_details"].append(cv_per_fold_status)
                    if len(final_co_train_clfs)!=0:
                        statistic_detail['Unlabel_size'].append(cv_per_fold_status[0]["unlabel_size"])
                        statistic_detail['Validation_size'].append(cv_per_fold_status[0]["validation_size"])
                        for clf, clf_name in final_co_train_clfs:
                            statistic_detail[clf_name+'_all_iterations'].append(int(cv_per_fold_status[0][clf_name+"_total_iteration"]))
                            statistic_detail[clf_name+'_total_self_labeled'].append(int(cv_per_fold_status[0][clf_name+"_total_self_labeled"]))
                            statistic_detail[clf_name+"_select_iteration"].append(int(cv_per_fold_status[0][clf_name+"_select_iteration"]))
                            statistic_detail[clf_name+"_k_iter_self_labeled"].append(int(cv_per_fold_status[0][clf_name+"_iter_k_self_labeled"]))

                    for clf_name, clf_f1 in final_f1_score:
                        statistic_detail[clf_name+"_f1"].append(clf_f1)
                        
                else:
                    print(name+ " is multi-class case, ignored")
                    
        # print(statistic_detail)
        print("Total number of selected group:",total_selected_group)
        print("Total number of selected binary group:",selected_binary_case_group)
        print("Total number of selected muti-class group:",(total_selected_group-selected_binary_case_group))
        # write evaluation result to excel
        output = pd.DataFrame(statistic_detail)
        print(output)
        #savePath = "../../result/"+Dataset+"/3_co_train_sample=140k/temp_cs=0.5/"
        #filename = "(init_labeled_size="+str(init_labeled_size)+")(Average of 100 run)(Fixed class ratio) V1="+v1_emb+"_V2="+v2_emb+"_threshold="+str(step_threshold)+".csv"
        #com_func.write_csv_df(savePath, filename, output)
        print("v1:",v1_emb," v2:",v2_emb, "threshold",step_threshold," Done")
        
        '''Save result with respect to threshold change'''
        threshold_change_all_method_f1s['Name'].append(statistic_detail['Name'])
        threshold_change_all_method_f1s['All_details'].append(statistic_detail["All_fold_details"])
        for col in output.columns: 
            if "f1" in col:
                threshold_change_all_method_f1s[col].append(statistic_detail[col])

    co_lr_diff_embedding_final_result["v1:"+v1_emb+" v2:"+v2_emb].append(threshold_change_all_method_f1s)


Load text embedding:  pv_dbow
Total text vector records: 135796
Vector dimension:  100
Load citation embedding:  n2v
Total citation vector records: 124922
Vector dimension:  100
For name:  j_read
(136, 2)
j_read  pass
For name:  f_esteves
(34, 2)
f_esteves  pass
For name:  c_miller
(252, 2)
c_miller  pass
For name:  r_jha
(11, 2)
r_jha  pass
For name:  a_lowe
(102, 2)
a_lowe  pass
For name:  a_vega
(20, 2)
a_vega  pass
For name:  k_smith
(338, 2)
k_smith  pass
For name:  j_gordon
(19, 2)
j_gordon  pass
For name:  s_liao
(104, 2)
s_liao  pass
For name:  j_qian
(17, 2)
j_qian  pass
For name:  s_bernardi
(91, 2)
s_bernardi  pass
For name:  t_hill
(15, 2)
t_hill  pass
For name:  s_schindler
(51, 2)
s_schindler  pass
For name:  j_williams
(625, 2)
j_williams  pass
For name:  s_jacobson
(28, 2)
s_jacobson  pass
For name:  e_andrade
(17, 2)
e_andrade  pass
For name:  t_santos
(45, 2)
t_santos  pass
For name:  k_kim
(1111, 2)
Total sample size before apply threshold:  1111
Counter({'0000-0002-

Total missing sample:  0
(252, 101)
Total missing sample:  6
(252, 101)
Papers with no citation:  [12, 61, 73, 140, 165, 168]
Total paper with citation:  246
For name:  c_zou
(32, 2)
c_zou  pass
For name:  s_rana
(42, 2)
s_rana  pass
For name:  a_nunes
(61, 2)
a_nunes  pass
For name:  s_jeong
(93, 2)
s_jeong  pass
For name:  b_olsen
(213, 2)
b_olsen  pass
For name:  m_reilly
(20, 2)
m_reilly  pass
For name:  d_nguyen
(25, 2)
d_nguyen  pass
For name:  r_santos
(184, 2)
r_santos  pass
For name:  f_ferreira
(224, 2)
f_ferreira  pass
For name:  y_ng
(19, 2)
y_ng  pass
For name:  j_madsen
(69, 2)
j_madsen  pass
For name:  d_collins
(31, 2)
d_collins  pass
For name:  l_davies
(96, 2)
l_davies  pass
For name:  m_mora
(131, 2)
m_mora  pass
For name:  a_fontana
(203, 2)
a_fontana  pass
For name:  r_chen
(367, 2)
r_chen  pass
For name:  s_krause
(70, 2)
s_krause  pass
For name:  t_smith
(603, 2)
Total sample size before apply threshold:  603
Counter({'0000-0002-3650-9381': 154, '0000-0003-1673-2

Total missing sample:  0
(238, 101)
Total missing sample:  11
(238, 101)
Papers with no citation:  [8, 24, 50, 54, 90, 92, 100, 147, 180, 198, 236]
Total paper with citation:  227
For name:  j_gao
(222, 2)
j_gao  pass
For name:  d_fernandes
(40, 2)
d_fernandes  pass
For name:  c_silva
(148, 2)
c_silva  pass
For name:  t_fitzgerald
(54, 2)
t_fitzgerald  pass
For name:  j_mitchell
(143, 2)
j_mitchell  pass
For name:  a_gomes
(244, 2)
a_gomes  pass
For name:  t_weber
(71, 2)
t_weber  pass
For name:  j_shim
(188, 2)
j_shim  pass
For name:  k_kang
(128, 2)
k_kang  pass
For name:  i_ferreira
(344, 2)
i_ferreira  pass
For name:  y_jia
(46, 2)
y_jia  pass
For name:  p_gaspar
(93, 2)
p_gaspar  pass
For name:  r_o'connor
(82, 2)
r_o'connor  pass
For name:  k_larsen
(47, 2)
k_larsen  pass
For name:  s_das
(197, 2)
s_das  pass
For name:  f_rodriguez
(6, 2)
f_rodriguez  pass
For name:  w_peng
(38, 2)
w_peng  pass
For name:  c_torres
(300, 2)
c_torres  pass
For name:  s_rossi
(199, 2)
s_rossi  pass


Total missing sample:  0
(292, 101)
Total missing sample:  7
(292, 101)
Papers with no citation:  [56, 104, 210, 247, 270, 272, 276]
Total paper with citation:  285
For name:  r_sinha
(27, 2)
r_sinha  pass
For name:  c_turner
(88, 2)
c_turner  pass
For name:  y_su
(190, 2)
y_su  pass
For name:  a_popov
(135, 2)
a_popov  pass
For name:  w_liao
(79, 2)
w_liao  pass
For name:  j_zhong
(280, 2)
j_zhong  pass
For name:  a_wheeler
(138, 2)
a_wheeler  pass
For name:  m_walsh
(37, 2)
m_walsh  pass
For name:  r_figueiredo
(48, 2)
r_figueiredo  pass
For name:  y_lin
(785, 2)
Total sample size before apply threshold:  785
Counter({'0000-0003-3791-7587': 146, '0000-0001-8153-1441': 115, '0000-0003-1224-6561': 64, '0000-0002-4192-3165': 49, '0000-0002-2499-8632': 39, '0000-0001-8667-0811': 33, '0000-0002-5887-0880': 24, '0000-0001-5227-2663': 23, '0000-0002-4350-7755': 23, '0000-0003-4913-8003': 22, '0000-0001-6460-2877': 21, '0000-0003-1954-334X': 20, '0000-0001-8572-649X': 20, '0000-0001-5574-706

Total missing sample:  0
(360, 101)
Total missing sample:  16
(360, 101)
Papers with no citation:  [13, 43, 94, 100, 129, 130, 143, 147, 185, 201, 249, 263, 272, 277, 321, 343]
Total paper with citation:  344
For name:  c_rodriguez
(43, 2)
c_rodriguez  pass
For name:  p_hall
(22, 2)
p_hall  pass
For name:  r_srivastava
(184, 2)
r_srivastava  pass
For name:  a_macedo
(29, 2)
a_macedo  pass
For name:  m_schultz
(40, 2)
m_schultz  pass
For name:  s_jacobs
(21, 2)
s_jacobs  pass
For name:  c_hong
(32, 2)
c_hong  pass
For name:  r_mohan
(7, 2)
r_mohan  pass
For name:  r_hill
(90, 2)
r_hill  pass
For name:  q_shen
(57, 2)
q_shen  pass
For name:  l_schmidt
(13, 2)
l_schmidt  pass
For name:  s_qin
(42, 2)
s_qin  pass
For name:  a_fabbri
(64, 2)
a_fabbri  pass
For name:  l_robinson
(93, 2)
l_robinson  pass
For name:  r_gross
(71, 2)
r_gross  pass
For name:  j_ahn
(130, 2)
j_ahn  pass
For name:  j_john
(43, 2)
j_john  pass
For name:  d_lloyd
(157, 2)
d_lloyd  pass
For name:  a_mohammadi
(8, 2)
a

(20, 2)
j_lynch  pass
For name:  j_boyle
(14, 2)
j_boyle  pass
For name:  r_turner
(147, 2)
r_turner  pass
For name:  s_brooks
(58, 2)
s_brooks  pass
For name:  p_moreira
(217, 2)
p_moreira  pass
For name:  s_mukhopadhyay
(119, 2)
s_mukhopadhyay  pass
For name:  a_hudson
(129, 2)
a_hudson  pass
For name:  d_thomas
(62, 2)
d_thomas  pass
For name:  w_smith
(61, 2)
w_smith  pass
For name:  l_martin
(253, 2)
l_martin  pass
For name:  c_garcia
(106, 2)
c_garcia  pass
For name:  g_huang
(160, 2)
g_huang  pass
For name:  j_huber
(96, 2)
j_huber  pass
For name:  j_qin
(96, 2)
j_qin  pass
For name:  t_ho
(83, 2)
t_ho  pass
For name:  c_keller
(15, 2)
c_keller  pass
For name:  m_maia
(99, 2)
m_maia  pass
For name:  p_bates
(154, 2)
p_bates  pass
For name:  s_chow
(29, 2)
s_chow  pass
For name:  m_simon
(66, 2)
m_simon  pass
For name:  s_kar
(36, 2)
s_kar  pass
For name:  d_vlachos
(101, 2)
d_vlachos  pass
For name:  e_law
(12, 2)
e_law  pass
For name:  m_ribeiro
(134, 2)
m_ribeiro  pass
For nam

Total missing sample:  0
(295, 101)
Total missing sample:  7
(295, 101)
Papers with no citation:  [13, 80, 109, 140, 208, 209, 221]
Total paper with citation:  288
For name:  r_ross
(374, 2)
r_ross  pass
For name:  k_yamamoto
(106, 2)
k_yamamoto  pass
For name:  j_silva
(268, 2)
j_silva  pass
For name:  m_pellegrini
(64, 2)
m_pellegrini  pass
For name:  s_kwon
(51, 2)
s_kwon  pass
For name:  m_correa
(72, 2)
m_correa  pass
For name:  a_pal
(14, 2)
a_pal  pass
For name:  v_costa
(141, 2)
v_costa  pass
For name:  j_allen
(111, 2)
j_allen  pass
For name:  y_dong
(76, 2)
y_dong  pass
For name:  m_fitzgerald
(133, 2)
m_fitzgerald  pass
For name:  m_ferreira
(253, 2)
m_ferreira  pass
For name:  m_roberts
(320, 2)
m_roberts  pass
For name:  y_lim
(76, 2)
y_lim  pass
For name:  g_miller
(76, 2)
g_miller  pass
For name:  x_kong
(69, 2)
x_kong  pass
For name:  w_cao
(126, 2)
w_cao  pass
For name:  c_ma
(126, 2)
c_ma  pass
For name:  j_chin
(27, 2)
j_chin  pass
For name:  h_kwon
(35, 2)
h_kwon  p

(150, 2)
m_ferrari  pass
For name:  j_paredes
(68, 2)
j_paredes  pass
For name:  z_zhao
(186, 2)
z_zhao  pass
For name:  j_cao
(39, 2)
j_cao  pass
For name:  d_kuo
(34, 2)
d_kuo  pass
For name:  a_andersen
(18, 2)
a_andersen  pass
For name:  m_longo
(44, 2)
m_longo  pass
For name:  h_chiang
(44, 2)
h_chiang  pass
For name:  m_o'brien
(34, 2)
m_o'brien  pass
For name:  s_ray
(123, 2)
s_ray  pass
For name:  a_cheng
(636, 2)
Total sample size before apply threshold:  636
Counter({'0000-0002-9152-6512': 265, '0000-0003-3152-116X': 180, '0000-0003-2345-6951': 71, '0000-0002-1182-7375': 38, '0000-0001-7897-4751': 29, '0000-0003-3862-2967': 25, '0000-0003-2729-606X': 22, '0000-0001-5137-000X': 2, '0000-0002-0977-0381': 2, '0000-0001-5196-3307': 1, '0000-0002-8166-0806': 1})
Total author before apply threshoid:  11
Total author after apply threshoid:  2
Total sample size after apply threshold:  445
Total missing sample:  0
(445, 101)
Total missing sample:  20
(445, 101)
Papers with no citation

For name:  s_o'brien
(32, 2)
s_o'brien  pass
For name:  c_webb
(35, 2)
c_webb  pass
For name:  c_adams
(69, 2)
c_adams  pass
For name:  c_peng
(103, 2)
c_peng  pass
For name:  k_kobayashi
(35, 2)
k_kobayashi  pass
For name:  s_larsen
(68, 2)
s_larsen  pass
Total number of selected group: 18
Total number of selected binary group: 0
Total number of selected muti-class group: 18
Empty DataFrame
Columns: []
Index: []
v1: pv_dbow  v2: n2v threshold 100  Done


In [ ]:
%tb

# Plots: 1. All method per fold variance 

In [ ]:
all_author_details = co_lr_diff_embedding_final_result['v1:pv_dbow v2:n2v'][0]['All_details'][0]

cotrain_all_per_fold_result= {}

for author in all_author_details:
    per_fold_f1 = collections.defaultdict(list)
    for per_fold_details in author:
        #print(per_fold_details)
        for key in per_fold_details:
            if key in ['co_LR f1','co_SVM f1','co_LR_SVM f1','Improved_co_LR f1','LR-LB f1','SVM-LB f1','LR-UB f1','SVM-UB f1']:
                per_fold_f1[key].append(per_fold_details[key])
    cotrain_all_per_fold_result[author[0]['author']]=(per_fold_f1)
#print(cotrain_all_per_fold_result)

In [ ]:
name_list = list(cotrain_all_per_fold_result.keys())
method_list = []
f1_mean = collections.defaultdict(list)
f1_min = collections.defaultdict(list)
f1_max = collections.defaultdict(list)
for author, author_result in cotrain_all_per_fold_result.items():
    #print(author)
    for method, method_result in author_result.items():
        if method.replace(' f1','') not in method_list:
            method_list.append(method.replace(' f1',''))
        f1_mean[method.replace(' f1','')].append(np.mean(method_result))
        f1_min[method.replace(' f1','')].append(np.min(method_result))
        f1_max[method.replace(' f1','')].append(np.max(method_result))
        #print(method_result, " mean: ", np.mean(method_result), " min: ",np.min(method_result)," max: ", np.max(method_result))
print(name_list)
print(method_list)

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

x = name_list
random_color = np.random.rand(len(method_list),3)

plt.figure(figsize=(15,10))

for idx, method in enumerate(method_list):
    y = f1_mean[method]
    y_min = f1_min[method]
    y_max = f1_max[method]
    
    plt.plot(x, y, 'k', color = random_color[idx], label=method)
    plt.fill_between(x, y_min, y_max, alpha=0.5, edgecolor = random_color[idx], facecolor=random_color[idx])
    plt.xticks(rotation=45, horizontalalignment='center')

legend = plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
plt.xlabel('Datasets')
plt.ylabel('F1 score')
#plt.savefig(plot_save_path+"/all_method_result_variance.png", dpi=150)
plt.show()

In [ ]:
x = name_list

fig, ax = plt.subplots(figsize=(15,10))

for idx, method in enumerate(method_list):
    y = f1_mean[method]
    y_min_error = [a - b for a, b in zip(f1_mean[method], f1_min[method])]
    y_max_error = [a - b for a, b in zip(f1_max[method], f1_mean[method])]
    y_error = [y_min_error,y_max_error]

    (_, caps, _) = ax.errorbar(x, y,yerr=y_error, fmt='-o',capsize=10, label=method)
    for cap in caps:
        cap.set_markeredgewidth(1)
    plt.xticks(rotation=45, horizontalalignment='center')

legend = plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
plt.xlabel('Datasets')
plt.ylabel('F1 score')
#plt.savefig(plot_save_path+"/all_method_result_variance.png", dpi=150)
plt.show()

# Plot: Each method per fold variance

In [ ]:
for idx, method in enumerate(method_list):
    plt.figure(figsize=(15,10))
    x = name_list
    y = f1_mean[method]
    y_min = f1_min[method]
    y_max = f1_max[method]
    
    plt.plot(x, y, 'k', color = random_color[idx],marker='o', label=method)
    plt.fill_between(x, y_min, y_max, alpha=0.5, edgecolor = random_color[idx], facecolor=random_color[idx])
    plt.xticks(rotation=45, horizontalalignment='center')
    legend = plt.legend()
    plt.xlabel('Datasets')
    plt.ylabel('F1 score')
    #plt.savefig(plot_save_path+"/"+method+"_result_variance.png", dpi=150)
    plt.show()

In [ ]:
for idx, method in enumerate(method_list):
    fig, ax = plt.subplots(figsize=(15,10))
    x = name_list
    y = f1_mean[method]
    y_min_error = [a - b for a, b in zip(f1_mean[method], f1_min[method])]
    y_max_error = [a - b for a, b in zip(f1_max[method], f1_mean[method])]
    y_error = [y_min_error,y_max_error]

    (_, caps, _) = ax.errorbar(x, y,yerr=y_error, fmt='-o',capsize=10, label=method)
    for cap in caps:
        cap.set_markeredgewidth(1)
    plt.xticks(rotation=45, horizontalalignment='center')
    legend = plt.legend()
    plt.xlabel('Datasets')
    plt.ylabel('F1 score')
    #plt.savefig(plot_save_path+"/"+method+"_result_variance.png", dpi=150)
    plt.show()

In [ ]:
#         # --------------- plot overall result f1 variance --------------- #
#         all_per_fold_f1_score_variance_plot = pd.DataFrame(all_per_fold_f1_score_variance)
#         ax = sns.boxplot(x="author", y="f1", data=all_per_fold_f1_score_variance_plot)
#         ax = sns.swarmplot(x="author", y="f1", data=all_per_fold_f1_score_variance_plot, color=".25")
#         plt.savefig(plot_save_path+"all_result_variance.png", dpi=300)
#         # plt.show()

# Test code parts

In [ ]:
import math
curr_iter_proba = [[0.99,0.01],[0.99,0.01],[0.99,0.01],[0.99,0.01],[0.99,0.01],
                   [0.01,0.99],[0.01,0.99],[0.01,0.99],[0.01,0.99],[0.01,0.99]]


# if one class have lots of high confident samples while other don't, change class ratio
class_ratio = []
for class_idx in range(2):
    high_confident = [proba for proba in curr_iter_proba if proba[class_idx]>0.95]
    high_confident_count = max(len(high_confident), 1)
    class_ratio.append(high_confident_count)
print(class_ratio)
for class_idx in range(2):
    class_new_sample_size = class_ratio[class_idx]
    if class_new_sample_size>5:
        class_new_sample_size=5
    print(class_new_sample_size)